## This code is used to analysis each model MACs info as well as the simulation results

In [47]:
import torch
from torch import nn
import torchvision
import torchvision.transforms as transforms
from d2l import torch as d2l
from torch.nn import functional as F
import numpy as np
import pandas as pd
from ptflops import get_model_complexity_info
from pathlib import Path
import subprocess
import os
import time

In [48]:
def fashionmnist_info(model):
    
    with torch.cuda.device(0):
        macs_f, params_f = get_model_complexity_info(model, (1, 224, 224), as_strings=True,
                                                print_per_layer_stat=True, verbose=True)
        print('{:<30}  {:<8}'.format('Computational complexity: ', macs_f))
        print('{:<30}  {:<8}'.format('Number of parameters: ', params_f))
    return macs_f, params_f

def cifar100_info(model):
    with torch.cuda.device(0):
        macs_c, params_c = get_model_complexity_info(model, (3, 224, 224), as_strings=True,
                                                print_per_layer_stat=True, verbose=True)
        print('{:<30}  {:<8}'.format('Computational complexity: ', macs_c))
        print('{:<30}  {:<8}'.format('Number of parameters: ', params_c))
    return macs_c, params_c

### Read all Models

#### Alexnet

In [49]:
def alexnet(img_channel, num_labels):
    net = nn.Sequential(
        # 这里使用一个11*11的更大窗口来捕捉对象。
        # 同时，步幅为4，以减少输出的高度和宽度。
        # 另外，输出通道的数目远大于LeNet
        nn.Conv2d(img_channel, 64, kernel_size=11, stride=4, padding=2), nn.ReLU(),
        nn.MaxPool2d(kernel_size=3, stride=2),
        # 减小卷积窗口，使用填充为2来使得输入与输出的高和宽一致，且增大输出通道数
        nn.Conv2d(64, 192, kernel_size=5, padding=2), nn.ReLU(),
        nn.MaxPool2d(kernel_size=3, stride=2),
        # 使用三个连续的卷积层和较小的卷积窗口。
        # 除了最后的卷积层，输出通道的数量进一步增加。
        # 在前两个卷积层之后，汇聚层不用于减少输入的高度和宽度
        nn.Conv2d(192, 384, kernel_size=3, padding=1), nn.ReLU(),
        nn.Conv2d(384, 256, kernel_size=3, padding=1), nn.ReLU(),
        nn.Conv2d(256, 256, kernel_size=3, padding=1), nn.ReLU(),
        nn.MaxPool2d(kernel_size=3, stride=2),
        nn.AdaptiveAvgPool2d((6, 6)),   # 使用全局平均池化对每个通道中所有元素求平均并直接将结果传递到全连接层
        nn.Flatten(),
        # 这里，全连接层的输出数量是LeNet中的好几倍。使用dropout层来减轻过拟合
        nn.Linear(256 * 6 * 6, 4096), nn.ReLU(),
        nn.Dropout(p=0.5),
        nn.Linear(4096, 4096), nn.ReLU(),
        nn.Dropout(p=0.5),
        # 最后是输出层。由于这里使用Fashion-MNIST，所以用类别数为10，而非论文中的1000
        nn.Linear(4096, num_labels))
    return net

# print the model structure
AlexNet = alexnet(1, 10)    
# print(net)
# print each layer
alexlayer = []
DNNlayer = []
num = 0
for layer in AlexNet:
    num += 1
    name = layer.__class__.__name__
    layer_name = name[:4] + str(num)
    DNNlayer.append(name)
    alexlayer.append(layer_name)
# find the unique layer name, and fix the order
DNNlayer_org = sorted(list(set(DNNlayer)))
print('The layer name is:', alexlayer)
print('The layer name after orged is:', DNNlayer_org)
# the number of layers, which contains ['AdaptiveAvgPool2d', 'Conv2d', 'Dropout', 'Flatten', 'Linear', 'MaxPool2d', 'ReLU']
alexlayer_num = len(alexlayer) 
print('The number of layers is:', alexlayer_num)

The layer name is: ['Conv1', 'ReLU2', 'MaxP3', 'Conv4', 'ReLU5', 'MaxP6', 'Conv7', 'ReLU8', 'Conv9', 'ReLU10', 'Conv11', 'ReLU12', 'MaxP13', 'Adap14', 'Flat15', 'Line16', 'ReLU17', 'Drop18', 'Line19', 'ReLU20', 'Drop21', 'Line22']
The layer name after orged is: ['AdaptiveAvgPool2d', 'Conv2d', 'Dropout', 'Flatten', 'Linear', 'MaxPool2d', 'ReLU']
The number of layers is: 22


In [52]:
AlexNet_f = alexnet(1, 10)
AlexNet_c = alexnet(3, 10)

AlexNet_f_modelinfo = []
AlexNet_c_modelinfo = []
# fashion mnist
macs_f, params_f = fashionmnist_info(AlexNet_f)
print('*'*50)
macs_c, params_c = cifar100_info(AlexNet_c)
    
AlexNet_f_modelinfo = [macs_f, params_f]
AlexNet_c_modelinfo = [macs_c, params_c]
print('The model info of AlexNet_f is:', AlexNet_f_modelinfo,'\n',
        'The model info of AlexNet_c is:', AlexNet_c_modelinfo)

Sequential(
  57.03 M, 100.000% Params, 664.65 MMac, 99.868% MACs, 
  (0): Conv2d(7.81 k, 0.014% Params, 23.62 MMac, 3.549% MACs, 1, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
  (1): ReLU(0, 0.000% Params, 193.6 KMac, 0.029% MACs, )
  (2): MaxPool2d(0, 0.000% Params, 193.6 KMac, 0.029% MACs, kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (3): Conv2d(307.39 k, 0.539% Params, 224.09 MMac, 33.671% MACs, 64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (4): ReLU(0, 0.000% Params, 139.97 KMac, 0.021% MACs, )
  (5): MaxPool2d(0, 0.000% Params, 139.97 KMac, 0.021% MACs, kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (6): Conv2d(663.94 k, 1.164% Params, 112.21 MMac, 16.859% MACs, 192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (7): ReLU(0, 0.000% Params, 64.9 KMac, 0.010% MACs, )
  (8): Conv2d(884.99 k, 1.552% Params, 149.56 MMac, 22.473% MACs, 384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (9): R

#### vgg11

In [53]:
def vgg_block(num_convs, in_channels, out_channels):
    layers = []
    for _ in range(num_convs):
        layers.append(nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1))
        layers.append(nn.ReLU())
        in_channels = out_channels
    layers.append(nn.MaxPool2d(kernel_size=2, stride=2))
    return nn.Sequential(*layers)

def vgg11(input_channels, output_channels):
    conv_arch = [(1, 64), (1, 128), (2, 256), (2, 512), (2, 512)]
    in_channels = input_channels  # For RGB images
    # Create convolutional layers
    conv_layers = []
    for num_convs, out_channels in conv_arch:
        conv_layers.append(vgg_block(num_convs, in_channels, out_channels))
        in_channels = out_channels

    return nn.Sequential(
        *conv_layers, nn.Flatten(),
        nn.Linear(out_channels * 7 * 7, 4096), nn.ReLU(), nn.Dropout(0.5),
        nn.Linear(4096, 4096), nn.ReLU(), nn.Dropout(0.5),
        nn.Linear(4096, output_channels)  # Output layer for 1000 classes
    )

Vgg11 = vgg11(1, 10)    
LayerName = []
block_num = 0
resblock_num = 0

for num, layer in Vgg11.named_children():  # 使用 named_children 来获取层名和层
    layername = layer.__class__.__name__
    # print(f"{num}: {layername}")  # 打印层名和层类名
    if layer.__class__.__name__ == 'Sequential':
        block_num += 1
        for sublayernum, sublayer in layer.named_children():  # 再次使用 named_children
            sublayername = sublayer.__class__.__name__
            if sublayername == 'Residual':
                resblock_num += 1
            # print(f"  {sublayernum}: {sublayername}")
            layer_label = f'{layername[0]}{num}_{sublayername[0]}{sublayernum}'
            LayerName.append(layer_label)  # 收集子块的类型
    else:
        layer_label = f'{layername[0]}{num}'
        LayerName.append(layer_label)
            
print('The layer name is:', LayerName)
print(f'The length of layer name is: {len(LayerName)}')
print('The number of blocks is:', block_num)
print('The number of inception blocks is:', resblock_num)

The layer name is: ['S0_C0', 'S0_R1', 'S0_M2', 'S1_C0', 'S1_R1', 'S1_M2', 'S2_C0', 'S2_R1', 'S2_C2', 'S2_R3', 'S2_M4', 'S3_C0', 'S3_R1', 'S3_C2', 'S3_R3', 'S3_M4', 'S4_C0', 'S4_R1', 'S4_C2', 'S4_R3', 'S4_M4', 'F5', 'L6', 'R7', 'D8', 'L9', 'R10', 'D11', 'L12']
The length of layer name is: 29
The number of blocks is: 5
The number of inception blocks is: 0


In [54]:
Vgg11_f = vgg11(1, 10)
Vgg11_c = vgg11(3, 100)

Vgg11_f_modelinfo = []
Vgg11_c_modelinfo = []

# fashion mnist
# with torch.cuda.device(0):
#     macs_f, params_f = get_model_complexity_info(Vgg11_f, (1, 224, 224), as_strings=True,
#                                             print_per_layer_stat=True, verbose=True)
#     print('{:<30}  {:<8}'.format('Computational complexity: ', macs_f))
#     print('{:<30}  {:<8}'.format('Number of parameters: ', params_f))
macs_f, params_f = fashionmnist_info(Vgg11_f)
print('*'*50)
macs_c, params_c = cifar100_info(Vgg11_c)

# cifar100
# with torch.cuda.device(0):
#     macs_c, params_c = get_model_complexity_info(Vgg11_c, (3, 224, 224), as_strings=True,
#                                             print_per_layer_stat=True, verbose=True)
#     print('{:<30}  {:<8}'.format('Computational complexity: ', macs_c))
#     print('{:<30}  {:<8}'.format('Number of parameters: ', params_c))
    
Vgg11_f_modelinfo = [macs_f, params_f]
Vgg11_c_modelinfo = [macs_c, params_c]

print('The model info of Vgg11_f is:', Vgg11_f_modelinfo,'\n',
        'The model info of Vgg11_c is:', Vgg11_c_modelinfo)

Sequential(
  128.81 M, 100.000% Params, 7.57 GMac, 99.821% MACs, 
  (0): Sequential(
    640, 0.000% Params, 38.54 MMac, 0.508% MACs, 
    (0): Conv2d(640, 0.000% Params, 32.11 MMac, 0.424% MACs, 1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(0, 0.000% Params, 3.21 MMac, 0.042% MACs, )
    (2): MaxPool2d(0, 0.000% Params, 3.21 MMac, 0.042% MACs, kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (1): Sequential(
    73.86 k, 0.057% Params, 929.66 MMac, 12.262% MACs, 
    (0): Conv2d(73.86 k, 0.057% Params, 926.45 MMac, 12.219% MACs, 64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(0, 0.000% Params, 1.61 MMac, 0.021% MACs, )
    (2): MaxPool2d(0, 0.000% Params, 1.61 MMac, 0.021% MACs, kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (2): Sequential(
    885.25 k, 0.687% Params, 2.78 GMac, 36.648% MACs, 
    (0): Conv2d(295.17 k, 0.229% Params, 925.65 MMac, 12.209% MACs, 128, 256, kernel_size=(3,

#### vgg16

In [55]:
def vgg_block(num_convs, in_channels, out_channels):
    layers = []
    for _ in range(num_convs):
        layers.append(nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1))
        layers.append(nn.ReLU())
        in_channels = out_channels
    layers.append(nn.MaxPool2d(kernel_size=2, stride=2))
    return nn.Sequential(*layers)

def vgg16(input_channels, output_channels):
    conv_arch = [(2, 64), (2, 128), (3, 256), (3, 512), (3, 512)]
    in_channels = input_channels  # For RGB images
    # Create convolutional layers
    conv_layers = []
    for num_convs, out_channels in conv_arch:
        conv_layers.append(vgg_block(num_convs, in_channels, out_channels))
        in_channels = out_channels

    return nn.Sequential(
        *conv_layers, nn.Flatten(),
        nn.Linear(out_channels * 7 * 7, 4096), nn.ReLU(), nn.Dropout(0.5),
        nn.Linear(4096, 4096), nn.ReLU(), nn.Dropout(0.5),
        nn.Linear(4096, output_channels)  # Output layer for 1000 classes
    )
    
Vgg16 = vgg16(1, 10)    
LayerName = []
block_num = 0
resblock_num = 0

for num, layer in Vgg16.named_children():  # 使用 named_children 来获取层名和层
    layername = layer.__class__.__name__
    # print(f"{num}: {layername}")  # 打印层名和层类名
    if layer.__class__.__name__ == 'Sequential':
        block_num += 1
        for sublayernum, sublayer in layer.named_children():  # 再次使用 named_children
            sublayername = sublayer.__class__.__name__
            if sublayername == 'Residual':
                resblock_num += 1
            # print(f"  {sublayernum}: {sublayername}")
            layer_label = f'{layername[0]}{num}_{sublayername[0]}{sublayernum}'
            LayerName.append(layer_label)  # 收集子块的类型
    else:
        layer_label = f'{layername[0]}{num}'
        LayerName.append(layer_label)
            
print('The layer name is:', LayerName)
print(f'The length of layer name is: {len(LayerName)}')
print('The number of blocks is:', block_num)
print('The number of inception blocks is:', resblock_num)

The layer name is: ['S0_C0', 'S0_R1', 'S0_C2', 'S0_R3', 'S0_M4', 'S1_C0', 'S1_R1', 'S1_C2', 'S1_R3', 'S1_M4', 'S2_C0', 'S2_R1', 'S2_C2', 'S2_R3', 'S2_C4', 'S2_R5', 'S2_M6', 'S3_C0', 'S3_R1', 'S3_C2', 'S3_R3', 'S3_C4', 'S3_R5', 'S3_M6', 'S4_C0', 'S4_R1', 'S4_C2', 'S4_R3', 'S4_C4', 'S4_R5', 'S4_M6', 'F5', 'L6', 'R7', 'D8', 'L9', 'R10', 'D11', 'L12']
The length of layer name is: 39
The number of blocks is: 5
The number of inception blocks is: 0


In [56]:
Vgg16_f = vgg16(1, 10)
Vgg16_c = vgg16(3, 100)

Vgg16_f_modelinfo = []
Vgg16_c_modelinfo = []

# fashion mnist
# with torch.cuda.device(0):
#     macs_f, params_f = get_model_complexity_info(Vgg16_f, (1, 224, 224), as_strings=True,
#                                             print_per_layer_stat=True, verbose=True)
#     print('{:<30}  {:<8}'.format('Computational complexity: ', macs_f))
#     print('{:<30}  {:<8}'.format('Number of parameters: ', params_f))
macs_f, params_f = fashionmnist_info(Vgg16_f)
print('*'*50)
macs_c, params_c = cifar100_info(Vgg16_c)

# cifar100
# with torch.cuda.device(0):
#     macs_c, params_c = get_model_complexity_info(Vgg16_c, (3, 224, 224), as_strings=True,
#                                             print_per_layer_stat=True, verbose=True)
#     print('{:<30}  {:<8}'.format('Computational complexity: ', macs_c))
#     print('{:<30}  {:<8}'.format('Number of parameters: ', params_c))
    
Vgg16_f_modelinfo = [macs_f, params_f]
Vgg16_c_modelinfo = [macs_c, params_c]

print('The model info of Vgg16_f is:', Vgg16_f_modelinfo,'\n',
        'The model info of Vgg16_c is:', Vgg16_c_modelinfo)

Sequential(
  134.3 M, 100.000% Params, 15.44 GMac, 99.873% MACs, 
  (0): Sequential(
    37.57 k, 0.028% Params, 1.89 GMac, 12.254% MACs, 
    (0): Conv2d(640, 0.000% Params, 32.11 MMac, 0.208% MACs, 1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(0, 0.000% Params, 3.21 MMac, 0.021% MACs, )
    (2): Conv2d(36.93 k, 0.027% Params, 1.85 GMac, 11.984% MACs, 64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(0, 0.000% Params, 3.21 MMac, 0.021% MACs, )
    (4): MaxPool2d(0, 0.000% Params, 3.21 MMac, 0.021% MACs, kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (1): Sequential(
    221.44 k, 0.165% Params, 2.78 GMac, 17.997% MACs, 
    (0): Conv2d(73.86 k, 0.055% Params, 926.45 MMac, 5.992% MACs, 64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(0, 0.000% Params, 1.61 MMac, 0.010% MACs, )
    (2): Conv2d(147.58 k, 0.110% Params, 1.85 GMac, 11.974% MACs, 128, 128, kernel_size=(3, 3), stride=(1, 1), pa

#### resnet18

In [57]:
class Residual(nn.Module):  #@save
    def __init__(self, input_channels, num_channels,
                 use_1x1conv=False, strides=1):
        super().__init__()
        self.conv1 = nn.Conv2d(input_channels, num_channels,
                               kernel_size=3, padding=1, stride=strides)
        self.conv2 = nn.Conv2d(num_channels, num_channels,
                               kernel_size=3, padding=1)
        if use_1x1conv:
            self.conv3 = nn.Conv2d(input_channels, num_channels,
                                   kernel_size=1, stride=strides)
        else:
            self.conv3 = None
        self.bn1 = nn.BatchNorm2d(num_channels)
        self.bn2 = nn.BatchNorm2d(num_channels)

    def forward(self, X):
        Y = F.relu(self.bn1(self.conv1(X)))
        Y = self.bn2(self.conv2(Y))
        if self.conv3:
            X = self.conv3(X)
        Y += X
        return F.relu(Y)
    
    
def resnet18(img_channel, num_labels):
    # blk = Residual(3,6, use_1x1conv=True, strides=2)

    b1 = nn.Sequential(nn.Conv2d(img_channel, 64, kernel_size=7, stride=2, padding=3),
                    nn.BatchNorm2d(64), nn.ReLU(),
                    nn.MaxPool2d(kernel_size=3, stride=2, padding=1))

    def resnet_block(input_channels, num_channels, num_residuals,
                    first_block=False):
        blk = []
        for i in range(num_residuals):
            if i == 0 and not first_block:
                blk.append(Residual(input_channels, num_channels,
                                    use_1x1conv=True, strides=2))
            else:
                blk.append(Residual(num_channels, num_channels))
        return blk

    b2 = nn.Sequential(*resnet_block(64, 64, 2, first_block=True))
    b3 = nn.Sequential(*resnet_block(64, 128, 2))
    b4 = nn.Sequential(*resnet_block(128, 256, 2))
    b5 = nn.Sequential(*resnet_block(256, 512, 2))

    net = nn.Sequential(b1, b2, b3, b4, b5,
                        nn.AdaptiveAvgPool2d((1,1)),
                        nn.Flatten(), nn.Linear(512, num_labels))
    return net

Resnet18 = resnet18(1, 10)    
LayerName = []
block_num = 0
resblock_num = 0

for num, layer in Resnet18.named_children():  # 使用 named_children 来获取层名和层
    layername = layer.__class__.__name__
    # print(f"{num}: {layername}")  # 打印层名和层类名
    if layer.__class__.__name__ == 'Sequential':
        block_num += 1
        for sublayernum, sublayer in layer.named_children():  # 再次使用 named_children
            sublayername = sublayer.__class__.__name__
            if sublayername == 'Residual':
                resblock_num += 1
            # print(f"  {sublayernum}: {sublayername}")
            layer_label = f'{layername[0]}{num}_{sublayername[0]}{sublayernum}'
            LayerName.append(layer_label)  # 收集子块的类型
    else:
        layer_label = f'{layername[0]}{num}'
        LayerName.append(layer_label)
            
print('The layer name is:', LayerName)
print(f'The length of layer name is: {len(LayerName)}')
print('The number of blocks is:', block_num)
print('The number of resnet blocks is:', resblock_num)

The layer name is: ['S0_C0', 'S0_B1', 'S0_R2', 'S0_M3', 'S1_R0', 'S1_R1', 'S2_R0', 'S2_R1', 'S3_R0', 'S3_R1', 'S4_R0', 'S4_R1', 'A5', 'F6', 'L7']
The length of layer name is: 15
The number of blocks is: 5
The number of resnet blocks is: 8


In [58]:
Resnet18_f = resnet18(1, 10)
Resnet18_c = resnet18(3, 100)

Resnet18_f_modelinfo = []
Resnet18_c_modelinfo = []

# fashion mnist
# with torch.cuda.device(0):
#     macs_f, params_f = get_model_complexity_info(Resnet18_f, (1, 224, 224), as_strings=True,
#                                             print_per_layer_stat=True, verbose=True)
#     print('{:<30}  {:<8}'.format('Computational complexity: ', macs_f))
#     print('{:<30}  {:<8}'.format('Number of parameters: ', params_f))

macs_f, params_f = fashionmnist_info(Resnet18_f)
print('*'*50)
macs_c, params_c = cifar100_info(Resnet18_c)

# cifar100
# with torch.cuda.device(0):
#     macs_c, params_c = get_model_complexity_info(Resnet18_c, (3, 224, 224), as_strings=True,
#                                             print_per_layer_stat=True, verbose=True)
#     print('{:<30}  {:<8}'.format('Computational complexity: ', macs_c))
#     print('{:<30}  {:<8}'.format('Number of parameters: ', params_c))
    
Resnet18_f_modelinfo = [macs_f, params_f]
Resnet18_c_modelinfo = [macs_c, params_c]

print('The model info of Resnet18_f is:', Resnet18_f_modelinfo,'\n',
        'The model info of Resnet18_c is:', Resnet18_c_modelinfo)

Sequential(
  11.18 M, 100.000% Params, 1.74 GMac, 99.820% MACs, 
  (0): Sequential(
    3.33 k, 0.030% Params, 43.35 MMac, 2.482% MACs, 
    (0): Conv2d(3.2 k, 0.029% Params, 40.14 MMac, 2.298% MACs, 1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
    (1): BatchNorm2d(128, 0.001% Params, 1.61 MMac, 0.092% MACs, 64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(0, 0.000% Params, 802.82 KMac, 0.046% MACs, )
    (3): MaxPool2d(0, 0.000% Params, 802.82 KMac, 0.046% MACs, kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  )
  (1): Sequential(
    148.22 k, 1.326% Params, 464.83 MMac, 26.611% MACs, 
    (0): Residual(
      74.11 k, 0.663% Params, 232.42 MMac, 13.306% MACs, 
      (conv1): Conv2d(36.93 k, 0.330% Params, 115.81 MMac, 6.630% MACs, 64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (conv2): Conv2d(36.93 k, 0.330% Params, 115.81 MMac, 6.630% MACs, 64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
 

#### resnet34

In [35]:
class Residual(nn.Module):  #@save
    def __init__(self, input_channels, num_channels,
                 use_1x1conv=False, strides=1):
        super().__init__()
        self.conv1 = nn.Conv2d(input_channels, num_channels,
                               kernel_size=3, padding=1, stride=strides)
        self.conv2 = nn.Conv2d(num_channels, num_channels,
                               kernel_size=3, padding=1)
        if use_1x1conv:
            self.conv3 = nn.Conv2d(input_channels, num_channels,
                                   kernel_size=1, stride=strides)
        else:
            self.conv3 = None
        self.bn1 = nn.BatchNorm2d(num_channels)
        self.bn2 = nn.BatchNorm2d(num_channels)

    def forward(self, X):
        Y = F.relu(self.bn1(self.conv1(X)))
        Y = self.bn2(self.conv2(Y))
        if self.conv3:
            X = self.conv3(X)
        Y += X
        return F.relu(Y)
    
    
def resnet34(img_channel, num_labels):
    # blk = Residual(3,6, use_1x1conv=True, strides=2)

    b1 = nn.Sequential(nn.Conv2d(img_channel, 64, kernel_size=7, stride=2, padding=3),
                    nn.BatchNorm2d(64), nn.ReLU(),
                    nn.MaxPool2d(kernel_size=3, stride=2, padding=1))

    def resnet_block(input_channels, num_channels, num_residuals,
                    first_block=False):
        blk = []
        for i in range(num_residuals):
            if i == 0 and not first_block:
                blk.append(Residual(input_channels, num_channels,
                                    use_1x1conv=True, strides=2))
            else:
                blk.append(Residual(num_channels, num_channels))
        return blk

    b2 = nn.Sequential(*resnet_block(64, 64, 3, first_block=True))
    b3 = nn.Sequential(*resnet_block(64, 128, 4))
    b4 = nn.Sequential(*resnet_block(128, 256, 6))
    b5 = nn.Sequential(*resnet_block(256, 512, 3))

    net = nn.Sequential(b1, b2, b3, b4, b5,
                        nn.AdaptiveAvgPool2d((1,1)),
                        nn.Flatten(), nn.Linear(512, num_labels))
    return net

Resnet34 = resnet34(1, 10)    
LayerName = []
block_num = 0
resblock_num = 0

for num, layer in Resnet34.named_children():  # 使用 named_children 来获取层名和层
    layername = layer.__class__.__name__
    # print(f"{num}: {layername}")  # 打印层名和层类名
    if layer.__class__.__name__ == 'Sequential':
        block_num += 1
        for sublayernum, sublayer in layer.named_children():  # 再次使用 named_children
            sublayername = sublayer.__class__.__name__
            if sublayername == 'Residual':
                resblock_num += 1
            # print(f"  {sublayernum}: {sublayername}")
            layer_label = f'{layername[0]}{num}_{sublayername[0]}{sublayernum}'
            LayerName.append(layer_label)  # 收集子块的类型
    else:
        layer_label = f'{layername[0]}{num}'
        LayerName.append(layer_label)
            
print('The layer name is:', LayerName)
print(f'The length of layer name is: {len(LayerName)}')
print('The number of blocks is:', block_num)
print('The number of resnet blocks is:', resblock_num)

The layer name is: ['S0_C0', 'S0_B1', 'S0_R2', 'S0_M3', 'S1_R0', 'S1_R1', 'S1_R2', 'S2_R0', 'S2_R1', 'S2_R2', 'S2_R3', 'S3_R0', 'S3_R1', 'S3_R2', 'S3_R3', 'S3_R4', 'S3_R5', 'S4_R0', 'S4_R1', 'S4_R2', 'A5', 'F6', 'L7']
The length of layer name is: 23
The number of blocks is: 5
The number of inception blocks is: 16


In [59]:
Resnet34_f = resnet34(1, 10)
Resnet34_c = resnet34(3, 100)

Resnet34_f_modelinfo = []
Resnet34_c_modelinfo = []

# fashion mnist
# with torch.cuda.device(0):
#     macs_f, params_f = get_model_complexity_info(Resnet34_f, (1, 224, 224), as_strings=True,
#                                             print_per_layer_stat=True, verbose=True)
#     print('{:<30}  {:<8}'.format('Computational complexity: ', macs_f))
#     print('{:<30}  {:<8}'.format('Number of parameters: ', params_f))

macs_f, params_f = fashionmnist_info(Resnet34_f)
print('*'*50)
macs_c, params_c = cifar100_info(Resnet34_c)

# cifar100
# with torch.cuda.device(0):
#     macs_c, params_c = get_model_complexity_info(Resnet34_c, (3, 224, 224), as_strings=True,
#                                             print_per_layer_stat=True, verbose=True)
#     print('{:<30}  {:<8}'.format('Computational complexity: ', macs_c))
#     print('{:<30}  {:<8}'.format('Number of parameters: ', params_c))
    
Resnet34_f_modelinfo = [macs_f, params_f]
Resnet34_c_modelinfo = [macs_c, params_c]

print('The model info of Resnet34_f is:', Resnet34_f_modelinfo,'\n',
        'The model info of Resnet34_c is:', Resnet34_c_modelinfo)

Sequential(
  21.29 M, 100.000% Params, 3.6 GMac, 99.878% MACs, 
  (0): Sequential(
    3.33 k, 0.016% Params, 43.35 MMac, 1.204% MACs, 
    (0): Conv2d(3.2 k, 0.015% Params, 40.14 MMac, 1.115% MACs, 1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
    (1): BatchNorm2d(128, 0.001% Params, 1.61 MMac, 0.045% MACs, 64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(0, 0.000% Params, 802.82 KMac, 0.022% MACs, )
    (3): MaxPool2d(0, 0.000% Params, 802.82 KMac, 0.022% MACs, kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  )
  (1): Sequential(
    222.34 k, 1.044% Params, 697.25 MMac, 19.360% MACs, 
    (0): Residual(
      74.11 k, 0.348% Params, 232.42 MMac, 6.453% MACs, 
      (conv1): Conv2d(36.93 k, 0.173% Params, 115.81 MMac, 3.216% MACs, 64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (conv2): Conv2d(36.93 k, 0.173% Params, 115.81 MMac, 3.216% MACs, 64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
   

#### googlenet

In [61]:
class Inception(nn.Module):
    # c1--c4是每条路径的输出通道数
    def __init__(self, in_channels, c1, c2, c3, c4, **kwargs):
        super(Inception, self).__init__(**kwargs)
        # 线路1，单1x1卷积层
        self.p1_1 = nn.Conv2d(in_channels, c1, kernel_size=1)
        # 线路2，1x1卷积层后接3x3卷积层
        self.p2_1 = nn.Conv2d(in_channels, c2[0], kernel_size=1)
        self.p2_2 = nn.Conv2d(c2[0], c2[1], kernel_size=3, padding=1)
        # 线路3，1x1卷积层后接5x5卷积层
        self.p3_1 = nn.Conv2d(in_channels, c3[0], kernel_size=1)
        self.p3_2 = nn.Conv2d(c3[0], c3[1], kernel_size=5, padding=2)
        # 线路4，3x3最大汇聚层后接1x1卷积层
        self.p4_1 = nn.MaxPool2d(kernel_size=3, stride=1, padding=1)
        self.p4_2 = nn.Conv2d(in_channels, c4, kernel_size=1)

    def forward(self, x):
        p1 = F.relu(self.p1_1(x))
        p2 = F.relu(self.p2_2(F.relu(self.p2_1(x))))
        p3 = F.relu(self.p3_2(F.relu(self.p3_1(x))))
        p4 = F.relu(self.p4_2(self.p4_1(x)))
        # 在通道维度上连结输出
        return torch.cat((p1, p2, p3, p4), dim=1)
    
def Googlenet(img_channel, num_labels):
    b1 = nn.Sequential(nn.Conv2d(img_channel, 64, kernel_size=7, stride=2, padding=3),
                   nn.ReLU(),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
    
    b2 = nn.Sequential(nn.Conv2d(64, 64, kernel_size=1),
                   nn.ReLU(),
                   nn.Conv2d(64, 192, kernel_size=3, padding=1),
                   nn.ReLU(),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
    
    b3 = nn.Sequential(Inception(192, 64, (96, 128), (16, 32), 32),
                   Inception(256, 128, (128, 192), (32, 96), 64),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
    
    b4 = nn.Sequential(Inception(480, 192, (96, 208), (16, 48), 64),
                   Inception(512, 160, (112, 224), (24, 64), 64),
                   Inception(512, 128, (128, 256), (24, 64), 64),
                   Inception(512, 112, (144, 288), (32, 64), 64),
                   Inception(528, 256, (160, 320), (32, 128), 128),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
    
    b5 = nn.Sequential(Inception(832, 256, (160, 320), (32, 128), 128),
                   Inception(832, 384, (192, 384), (48, 128), 128),
                   nn.AdaptiveAvgPool2d((1,1)),
                   nn.Flatten())

    net = nn.Sequential(b1, b2, b3, b4, b5, nn.Linear(1024, num_labels))
    return net

googlenet = Googlenet(1, 10)    
LayerName = []
block_num = 0
incep_num = 0

for num, layer in googlenet.named_children():  # 使用 named_children 来获取层名和层
    layername = layer.__class__.__name__
    # print(f"{num}: {layername}")  # 打印层名和层类名
    
    if layer.__class__.__name__ == 'Sequential':
        block_num += 1
        for sublayernum, sublayer in layer.named_children():  # 再次使用 named_children
            sublayername = sublayer.__class__.__name__
            if sublayername == 'Inception':
                incep_num += 1
            # print(f"  {sublayernum}: {sublayername}")
            layer_label = f'{layername[0]}{num}_{sublayername[0]}{sublayernum}'
            LayerName.append(layer_label)  # 收集子块的类型
    else:
        layer_label = f'{layername[0]}{num}'
        LayerName.append(layer_label)  # 收集块的类型
                        
print('The layer name is:', LayerName)
print(f'The length of layer name is: {len(LayerName)}')
print('The number of blocks is:', block_num)
print('The number of inception blocks is:', incep_num)

The layer name is: ['S0_C0', 'S0_R1', 'S0_M2', 'S1_C0', 'S1_R1', 'S1_C2', 'S1_R3', 'S1_M4', 'S2_I0', 'S2_I1', 'S2_M2', 'S3_I0', 'S3_I1', 'S3_I2', 'S3_I3', 'S3_I4', 'S3_M5', 'S4_I0', 'S4_I1', 'S4_A2', 'S4_F3', 'L5']
The length of layer name is: 22
The number of blocks is: 5
The number of inception blocks is: 9


In [62]:
googlenet_f = Googlenet(1, 10)
googlenet_c = Googlenet(3, 100)

googlenet_f_modelinfo = []
googlenet_c_modelinfo = []

# fashion mnist
# with torch.cuda.device(0):
#     macs_f, params_f = get_model_complexity_info(googlenet_f, (1, 224, 224), as_strings=True,
#                                             print_per_layer_stat=True, verbose=True)
#     print('{:<30}  {:<8}'.format('Computational complexity: ', macs_f))
#     print('{:<30}  {:<8}'.format('Number of parameters: ', params_f))

macs_f, params_f = fashionmnist_info(googlenet_f)
print('*'*50)
macs_c, params_c = cifar100_info(googlenet_c)

# cifar100
# with torch.cuda.device(0):
#     macs_c, params_c = get_model_complexity_info(googlenet_c, (3, 224, 224), as_strings=True,
#                                             print_per_layer_stat=True, verbose=True)
#     print('{:<30}  {:<8}'.format('Computational complexity: ', macs_c))
#     print('{:<30}  {:<8}'.format('Number of parameters: ', params_c))
    
googlenet_f_modelinfo = [macs_f, params_f]
googlenet_c_modelinfo = [macs_c, params_c]

print('The model info of googlenet_f is:', googlenet_f_modelinfo,'\n',
        'The model info of googlenet_c is:', googlenet_c_modelinfo)

Sequential(
  5.98 M, 100.000% Params, 1.51 GMac, 99.594% MACs, 
  (0): Sequential(
    3.2 k, 0.054% Params, 41.75 MMac, 2.752% MACs, 
    (0): Conv2d(3.2 k, 0.054% Params, 40.14 MMac, 2.646% MACs, 1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
    (1): ReLU(0, 0.000% Params, 802.82 KMac, 0.053% MACs, )
    (2): MaxPool2d(0, 0.000% Params, 802.82 KMac, 0.053% MACs, kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  )
  (1): Sequential(
    114.94 k, 1.923% Params, 361.87 MMac, 23.856% MACs, 
    (0): Conv2d(4.16 k, 0.070% Params, 13.05 MMac, 0.860% MACs, 64, 64, kernel_size=(1, 1), stride=(1, 1))
    (1): ReLU(0, 0.000% Params, 200.7 KMac, 0.013% MACs, )
    (2): Conv2d(110.78 k, 1.853% Params, 347.42 MMac, 22.903% MACs, 64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(0, 0.000% Params, 602.11 KMac, 0.040% MACs, )
    (4): MaxPool2d(0, 0.000% Params, 602.11 KMac, 0.040% MACs, kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=

#### googlenet_mod0
googlenet with no inception block added

In [64]:
# class Inception_mod8(nn.Module):
#     # c1--c4是每条路径的输出通道数
#     def __init__(self, in_channels, c2, **kwargs):
#         super(Inception_mod8, self).__init__(**kwargs)
#         # 线路1，单1x1卷积层
#         self.p1_1 = nn.Conv2d(in_channels, c2[0], kernel_size=1)
#         self.p1_2 = nn.Conv2d(c2[0], c2[1], kernel_size=3, padding=1)
#         # 线路2，1x1卷积层后接3x3卷积层
#         self.p2_1 = nn.Conv2d(in_channels, c2[0], kernel_size=1)
#         self.p2_2 = nn.Conv2d(c2[0], c2[1], kernel_size=3, padding=1)
#         # 线路3，1x1卷积层后接5x5卷积层
#         self.p3_1 = nn.Conv2d(in_channels, c2[0], kernel_size=1)
#         self.p3_2 = nn.Conv2d(c2[0], c2[1], kernel_size=3, padding=1)
#         # 线路4，3x3最大汇聚层后接1x1卷积层
#         self.p4_1 = nn.Conv2d(in_channels, c2[0], kernel_size=1)
#         self.p4_2 = nn.Conv2d(c2[0], c2[1], kernel_size=3, padding=1)

#     def forward(self, x):
#         p1 = F.relu(self.p1_2(F.relu(self.p1_1(x))))
#         p2 = F.relu(self.p2_2(F.relu(self.p2_1(x))))
#         p3 = F.relu(self.p3_2(F.relu(self.p3_1(x))))
#         p4 = F.relu(self.p4_2(F.relu(self.p4_1(x))))
#         # 在通道维度上连结输出
#         return torch.cat((p1, p2, p3, p4), dim=1)
    
def Googlenet_mod0(img_channel, num_labels):
    b1 = nn.Sequential(nn.Conv2d(img_channel, 64, kernel_size=7, stride=2, padding=3),
                   nn.ReLU(),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
    
    b2 = nn.Sequential(nn.Conv2d(64, 64, kernel_size=1),
                   nn.ReLU(),
                   nn.Conv2d(64, 192, kernel_size=3, padding=1),
                   nn.ReLU(),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
    
    # b3 = nn.Sequential(Inception_mod8(192, (96, 128)),
    #                Inception_mod8(128*4, (128, 192)),
    #                nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
    
    # b4 = nn.Sequential(Inception_mod8(192*4, (96, 208)),
    #                Inception_mod8(208*4, (112, 224)),
    #                Inception_mod8(224*4, (128, 256)),
    #                Inception_mod8(256*4, (144, 288)),
    #                Inception_mod8(288*4, (160, 320)),
    #                nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
    
    # b5 = nn.Sequential(Inception_mod8(320*4, (160, 320)),
    #                Inception_mod8(320*4, (192, 384)),
    #                nn.AdaptiveAvgPool2d((1,1)),
    #                nn.Flatten())
    
    b5 = nn.Sequential(
                   nn.AdaptiveAvgPool2d((1,1)),
                   nn.Flatten())

    net = nn.Sequential(b1, b2, b5, nn.Linear(192, num_labels))
    return net

googlenet0 = Googlenet_mod0(1, 10) 
LayerName = []
block_num = 0
incep_num = 0

for num, layer in googlenet0.named_children():  # 使用 named_children 来获取层名和层
    layername = layer.__class__.__name__
    # print(f"{num}: {layername}")  # 打印层名和层类名
    
    if layer.__class__.__name__ == 'Sequential':
        block_num += 1
        for sublayernum, sublayer in layer.named_children():  # 再次使用 named_children
            sublayername = sublayer.__class__.__name__
            if sublayername == 'Inception_mod5':
                incep_num += 1
            # print(f"  {sublayernum}: {sublayername}")
            layer_label = f'{layername[0]}{num}_{sublayername[0]}{sublayernum}'
            LayerName.append(layer_label)  # 收集子块的类型
                        
print('The layer name is:', LayerName)
print(f'The length of layer name is: {len(LayerName)}')
print('The number of blocks is:', block_num)
print('The number of inception blocks is:', incep_num)

The layer name is: ['S0_C0', 'S0_R1', 'S0_M2', 'S1_C0', 'S1_R1', 'S1_C2', 'S1_R3', 'S1_M4', 'S2_A0', 'S2_F1']
The length of layer name is: 10
The number of blocks is: 3
The number of inception blocks is: 0


In [65]:
googlenet_fmod0 = Googlenet_mod0(1, 10)
googlenet_cmod0 = Googlenet_mod0(3, 100)

googlenet_fmod0_modelinfo = []
googlenet_cmod0_modelinfo = []

# print('*'*50)
# print('The model complexity of googlenet_fmod0:')
# with torch.cuda.device(0):
#     macs_f, params_f = get_model_complexity_info(googlenet_fmod0, (1, 224, 224), as_strings=True,
#                                             print_per_layer_stat=True, verbose=True)
#     print('{:<30}  {:<8}'.format('Computational complexity: ', macs_f))
#     print('{:<30}  {:<8}'.format('Number of parameters: ', params_f))

macs_f, params_f = fashionmnist_info(googlenet_fmod0)
print('*'*50)
macs_c, params_c = cifar100_info(googlenet_cmod0)

# print('The model complexity of googlenet_cmod0:')
# with torch.cuda.device(0):
#     macs_c, params_c = get_model_complexity_info(googlenet_cmod0, (3, 224, 224), as_strings=True,
#                                             print_per_layer_stat=True, verbose=True)
#     print('{:<30}  {:<8}'.format('Computational complexity: ', macs_c))
#     print('{:<30}  {:<8}'.format('Number of parameters: ', params_c))
    
googlenet_fmod0_modelinfo = [macs_f, params_f]
googlenet_cmod0_modelinfo = [macs_c, params_c]

print('The model info of googlenet_fmod0 is:', googlenet_fmod0_modelinfo,'\n',
        'The model info of googlenet_cmod0 is:', googlenet_cmod0_modelinfo)

Sequential(
  120.07 k, 100.000% Params, 403.77 MMac, 99.223% MACs, 
  (0): Sequential(
    3.2 k, 2.665% Params, 41.75 MMac, 10.259% MACs, 
    (0): Conv2d(3.2 k, 2.665% Params, 40.14 MMac, 9.864% MACs, 1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
    (1): ReLU(0, 0.000% Params, 802.82 KMac, 0.197% MACs, )
    (2): MaxPool2d(0, 0.000% Params, 802.82 KMac, 0.197% MACs, kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  )
  (1): Sequential(
    114.94 k, 95.728% Params, 361.87 MMac, 88.927% MACs, 
    (0): Conv2d(4.16 k, 3.465% Params, 13.05 MMac, 3.206% MACs, 64, 64, kernel_size=(1, 1), stride=(1, 1))
    (1): ReLU(0, 0.000% Params, 200.7 KMac, 0.049% MACs, )
    (2): Conv2d(110.78 k, 92.263% Params, 347.42 MMac, 85.376% MACs, 64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(0, 0.000% Params, 602.11 KMac, 0.148% MACs, )
    (4): MaxPool2d(0, 0.000% Params, 602.11 KMac, 0.148% MACs, kernel_size=3, stride=2, padding=1, dilation=1, cei

#### googlenet_mod1
all the inception block is the same as the route2, with only 1 line in the inception block

In [66]:
class Inception_mod5(nn.Module):
    # c1--c4是每条路径的输出通道数
    def __init__(self, in_channels, c2, **kwargs):
        super(Inception_mod5, self).__init__(**kwargs)
        # 线路1，单1x1卷积层
        # self.p1_1 = nn.Conv2d(in_channels, c1, kernel_size=1)
        # 线路2，1x1卷积层后接3x3卷积层
        self.p2_1 = nn.Conv2d(in_channels, c2[0], kernel_size=1)
        self.p2_2 = nn.Conv2d(c2[0], c2[1], kernel_size=3, padding=1)
        # 线路3，1x1卷积层后接5x5卷积层
        # self.p3_1 = nn.Conv2d(in_channels, c3[0], kernel_size=1)
        # self.p3_2 = nn.Conv2d(c3[0], c3[1], kernel_size=5, padding=2)
        # 线路4，3x3最大汇聚层后接1x1卷积层
        # self.p4_1 = nn.MaxPool2d(kernel_size=3, stride=1, padding=1)
        # self.p4_2 = nn.Conv2d(in_channels, c4, kernel_size=1)

    def forward(self, x):
        # p1 = F.relu(self.p1_1(x))
        p2 = F.relu(self.p2_2(F.relu(self.p2_1(x))))
        # p3 = F.relu(self.p3_2(F.relu(self.p3_1(x))))
        # p4 = F.relu(self.p4_2(self.p4_1(x)))
        # 在通道维度上连结输出
        return torch.cat([p2], dim=1)
    
def Googlenet_mod1(img_channel, num_labels):
    b1 = nn.Sequential(nn.Conv2d(img_channel, 64, kernel_size=7, stride=2, padding=3),
                   nn.ReLU(),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
    
    b2 = nn.Sequential(nn.Conv2d(64, 64, kernel_size=1),
                   nn.ReLU(),
                   nn.Conv2d(64, 192, kernel_size=3, padding=1),
                   nn.ReLU(),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
    
    b3 = nn.Sequential(Inception_mod5(192, (96, 128)),
                   Inception_mod5(128, (128, 192)),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
    
    b4 = nn.Sequential(Inception_mod5(192, (96, 208)),
                   Inception_mod5(208, (112, 224)),
                   Inception_mod5(224, (128, 256)),
                   Inception_mod5(256, (144, 288)),
                   Inception_mod5(288, (160, 320)),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
    
    b5 = nn.Sequential(Inception_mod5(320, (160, 320)),
                   Inception_mod5(320, (192, 384)),
                   nn.AdaptiveAvgPool2d((1,1)),
                   nn.Flatten())

    net = nn.Sequential(b1, b2, b3, b4, b5, nn.Linear(384, num_labels))
    return net

googlenet_mod1 = Googlenet_mod1(1, 10) 
LayerName = []
block_num = 0
incep_num = 0

for num, layer in googlenet_mod1.named_children():  # 使用 named_children 来获取层名和层
    layername = layer.__class__.__name__
    # print(f"{num}: {layername}")  # 打印层名和层类名
    
    if layer.__class__.__name__ == 'Sequential':
        block_num += 1
        for sublayernum, sublayer in layer.named_children():  # 再次使用 named_children
            sublayername = sublayer.__class__.__name__
            if sublayername == 'Inception_mod5':
                incep_num += 1
            # print(f"  {sublayernum}: {sublayername}")
            layer_label = f'{layername[0]}{num}_{sublayername[0]}{sublayernum}'
            LayerName.append(layer_label)  # 收集子块的类型
                        
print('The layer name is:', LayerName)
print(f'The length of layer name is: {len(LayerName)}')
print('The number of blocks is:', block_num)
print('The number of inception blocks is:', incep_num)

The layer name is: ['S0_C0', 'S0_R1', 'S0_M2', 'S1_C0', 'S1_R1', 'S1_C2', 'S1_R3', 'S1_M4', 'S2_I0', 'S2_I1', 'S2_M2', 'S3_I0', 'S3_I1', 'S3_I2', 'S3_I3', 'S3_I4', 'S3_M5', 'S4_I0', 'S4_I1', 'S4_A2', 'S4_F3']
The length of layer name is: 21
The number of blocks is: 5
The number of inception blocks is: 9


In [67]:
googlenet_fmod1 = Googlenet_mod1(1, 10)
googlenet_cmod1 = Googlenet_mod1(3, 100)

googlenet_fmod1_modelinfo = []
googlenet_cmod1_modelinfo = []

# print('The model complexity of googlenet_fmod1:')
# with torch.cuda.device(0):
#     macs_f, params_f = get_model_complexity_info(googlenet_fmod1, (1, 224, 224), as_strings=True,
#                                             print_per_layer_stat=True, verbose=True)
#     print('{:<30}  {:<8}'.format('Computational complexity: ', macs_f))
#     print('{:<30}  {:<8}'.format('Number of parameters: ', params_f))
    
macs_f, params_f = fashionmnist_info(googlenet_fmod1)   
print('*'*50)
macs_c, params_c = cifar100_info(googlenet_cmod1)

# print('The model complexity of googlenet_cmod5:')
# with torch.cuda.device(0):
#     macs_c, params_c = get_model_complexity_info(googlenet_cmod1, (3, 224, 224), as_strings=True,
#                                             print_per_layer_stat=True, verbose=True)
#     print('{:<30}  {:<8}'.format('Computational complexity: ', macs_c))
#     print('{:<30}  {:<8}'.format('Number of parameters: ', params_c))
    
googlenet_fmod1_modelinfo = [macs_f, params_f]
googlenet_cmod1_modelinfo = [macs_c, params_c]

print('The model info of googlenet_fmod1 is:', googlenet_fmod1_modelinfo,'\n',
        'The model info of googlenet_cmod1 is:', googlenet_cmod1_modelinfo)

Sequential(
  3.42 M, 100.000% Params, 1.08 GMac, 99.623% MACs, 
  (0): Sequential(
    3.2 k, 0.094% Params, 41.75 MMac, 3.838% MACs, 
    (0): Conv2d(3.2 k, 0.094% Params, 40.14 MMac, 3.691% MACs, 1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
    (1): ReLU(0, 0.000% Params, 802.82 KMac, 0.074% MACs, )
    (2): MaxPool2d(0, 0.000% Params, 802.82 KMac, 0.074% MACs, kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  )
  (1): Sequential(
    114.94 k, 3.364% Params, 361.87 MMac, 33.271% MACs, 
    (0): Conv2d(4.16 k, 0.122% Params, 13.05 MMac, 1.199% MACs, 64, 64, kernel_size=(1, 1), stride=(1, 1))
    (1): ReLU(0, 0.000% Params, 200.7 KMac, 0.018% MACs, )
    (2): Conv2d(110.78 k, 3.242% Params, 347.42 MMac, 31.942% MACs, 64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(0, 0.000% Params, 602.11 KMac, 0.055% MACs, )
    (4): MaxPool2d(0, 0.000% Params, 602.11 KMac, 0.055% MACs, kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=

#### googlenet_mod2
all the inception block is the same as the route2, with only 2 line in the inception block

In [69]:
class Inception_mod6(nn.Module):
    # c1--c4是每条路径的输出通道数
    def __init__(self, in_channels, c2, **kwargs):
        super(Inception_mod6, self).__init__(**kwargs)
        # 线路1，单1x1卷积层
        # self.p1_1 = nn.Conv2d(in_channels, c1, kernel_size=1)
        # 线路2，1x1卷积层后接3x3卷积层
        self.p2_1 = nn.Conv2d(in_channels, c2[0], kernel_size=1)
        self.p2_2 = nn.Conv2d(c2[0], c2[1], kernel_size=3, padding=1)
        # 线路3，1x1卷积层后接5x5卷积层
        self.p3_1 = nn.Conv2d(in_channels, c2[0], kernel_size=1)
        self.p3_2 = nn.Conv2d(c2[0], c2[1], kernel_size=3, padding=1)
        # 线路4，3x3最大汇聚层后接1x1卷积层
        # self.p4_1 = nn.MaxPool2d(kernel_size=3, stride=1, padding=1)
        # self.p4_2 = nn.Conv2d(in_channels, c4, kernel_size=1)

    def forward(self, x):
        # p1 = F.relu(self.p1_1(x))
        p2 = F.relu(self.p2_2(F.relu(self.p2_1(x))))
        p3 = F.relu(self.p3_2(F.relu(self.p3_1(x))))
        # p4 = F.relu(self.p4_2(self.p4_1(x)))
        # 在通道维度上连结输出
        return torch.cat((p2, p3), dim=1)
    
def Googlenet_mod2(img_channel, num_labels):
    b1 = nn.Sequential(nn.Conv2d(img_channel, 64, kernel_size=7, stride=2, padding=3),
                   nn.ReLU(),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
    
    b2 = nn.Sequential(nn.Conv2d(64, 64, kernel_size=1),
                   nn.ReLU(),
                   nn.Conv2d(64, 192, kernel_size=3, padding=1),
                   nn.ReLU(),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
    
    b3 = nn.Sequential(Inception_mod6(192, (96, 128)),
                   Inception_mod6(128*2, (128, 192)),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
    
    b4 = nn.Sequential(Inception_mod6(192*2, (96, 208)),
                   Inception_mod6(208*2, (112, 224)),
                   Inception_mod6(224*2, (128, 256)),
                   Inception_mod6(256*2, (144, 288)),
                   Inception_mod6(288*2, (160, 320)),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
    
    b5 = nn.Sequential(Inception_mod6(320*2, (160, 320)),
                   Inception_mod6(320*2, (192, 384)),
                   nn.AdaptiveAvgPool2d((1,1)),
                   nn.Flatten())

    net = nn.Sequential(b1, b2, b3, b4, b5, nn.Linear(384*2, num_labels))
    return net

googlenet2 = Googlenet_mod2(1, 10) 
LayerName = []
block_num = 0
incep_num = 0

for num, layer in googlenet2.named_children():  # 使用 named_children 来获取层名和层
    layername = layer.__class__.__name__
    # print(f"{num}: {layername}")  # 打印层名和层类名
    
    if layer.__class__.__name__ == 'Sequential':
        block_num += 1
        for sublayernum, sublayer in layer.named_children():  # 再次使用 named_children
            sublayername = sublayer.__class__.__name__
            if sublayername == 'Inception_mod5':
                incep_num += 1
            # print(f"  {sublayernum}: {sublayername}")
            layer_label = f'{layername[0]}{num}_{sublayername[0]}{sublayernum}'
            LayerName.append(layer_label)  # 收集子块的类型
                        
print('The layer name is:', LayerName)
print(f'The length of layer name is: {len(LayerName)}')
print('The number of blocks is:', block_num)
print('The number of inception blocks is:', incep_num)

The layer name is: ['S0_C0', 'S0_R1', 'S0_M2', 'S1_C0', 'S1_R1', 'S1_C2', 'S1_R3', 'S1_M4', 'S2_I0', 'S2_I1', 'S2_M2', 'S3_I0', 'S3_I1', 'S3_I2', 'S3_I3', 'S3_I4', 'S3_M5', 'S4_I0', 'S4_I1', 'S4_A2', 'S4_F3']
The length of layer name is: 21
The number of blocks is: 5
The number of inception blocks is: 0


In [70]:
googlenet_fmod2 = Googlenet_mod2(1, 10)
googlenet_cmod2 = Googlenet_mod2(3, 100)

googlenet_fmod2_modelinfo = []
googlenet_cmod2_modelinfo = []

macs_f, params_f = fashionmnist_info(googlenet_fmod2)   
print('*'*50)
macs_c, params_c = cifar100_info(googlenet_cmod2)

googlenet_fmod2_modelinfo = [macs_f, params_f]
googlenet_cmod2_modelinfo = [macs_c, params_c]

print('The model info of googlenet_fmod2 is:', googlenet_fmod2_modelinfo,'\n',
        'The model info of googlenet_cmod2 is:', googlenet_cmod2_modelinfo)

Sequential(
  7.28 M, 100.000% Params, 1.86 GMac, 99.722% MACs, 
  (0): Sequential(
    3.2 k, 0.044% Params, 41.75 MMac, 2.238% MACs, 
    (0): Conv2d(3.2 k, 0.044% Params, 40.14 MMac, 2.152% MACs, 1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
    (1): ReLU(0, 0.000% Params, 802.82 KMac, 0.043% MACs, )
    (2): MaxPool2d(0, 0.000% Params, 802.82 KMac, 0.043% MACs, kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  )
  (1): Sequential(
    114.94 k, 1.579% Params, 361.87 MMac, 19.398% MACs, 
    (0): Conv2d(4.16 k, 0.057% Params, 13.05 MMac, 0.699% MACs, 64, 64, kernel_size=(1, 1), stride=(1, 1))
    (1): ReLU(0, 0.000% Params, 200.7 KMac, 0.011% MACs, )
    (2): Conv2d(110.78 k, 1.522% Params, 347.42 MMac, 18.623% MACs, 64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(0, 0.000% Params, 602.11 KMac, 0.032% MACs, )
    (4): MaxPool2d(0, 0.000% Params, 602.11 KMac, 0.032% MACs, kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=

#### googlenet_mod3
all the inception block is the same as the route2, with only 3 line in the inception block

In [71]:
class Inception_mod7(nn.Module):
    # c1--c4是每条路径的输出通道数
    def __init__(self, in_channels, c2, **kwargs):
        super(Inception_mod7, self).__init__(**kwargs)
        # 线路1，单1x1卷积层
        # self.p1_1 = nn.Conv2d(in_channels, c1, kernel_size=1)
        # 线路2，1x1卷积层后接3x3卷积层
        self.p2_1 = nn.Conv2d(in_channels, c2[0], kernel_size=1)
        self.p2_2 = nn.Conv2d(c2[0], c2[1], kernel_size=3, padding=1)
        # 线路3，1x1卷积层后接5x5卷积层
        self.p3_1 = nn.Conv2d(in_channels, c2[0], kernel_size=1)
        self.p3_2 = nn.Conv2d(c2[0], c2[1], kernel_size=3, padding=1)
        # 线路4，3x3最大汇聚层后接1x1卷积层
        self.p4_1 = nn.Conv2d(in_channels, c2[0], kernel_size=1)
        self.p4_2 = nn.Conv2d(c2[0], c2[1], kernel_size=3, padding=1)

    def forward(self, x):
        # p1 = F.relu(self.p1_1(x))
        p2 = F.relu(self.p2_2(F.relu(self.p2_1(x))))
        p3 = F.relu(self.p3_2(F.relu(self.p3_1(x))))
        p4 = F.relu(self.p4_2(F.relu(self.p4_1(x))))
        # 在通道维度上连结输出
        return torch.cat((p2, p3, p4), dim=1)
    
def Googlenet_mod3(img_channel, num_labels):
    b1 = nn.Sequential(nn.Conv2d(img_channel, 64, kernel_size=7, stride=2, padding=3),
                   nn.ReLU(),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
    
    b2 = nn.Sequential(nn.Conv2d(64, 64, kernel_size=1),
                   nn.ReLU(),
                   nn.Conv2d(64, 192, kernel_size=3, padding=1),
                   nn.ReLU(),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
    
    b3 = nn.Sequential(Inception_mod7(192, (96, 128)),
                   Inception_mod7(128*3, (128, 192)),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
    
    b4 = nn.Sequential(Inception_mod7(192*3, (96, 208)),
                   Inception_mod7(208*3, (112, 224)),
                   Inception_mod7(224*3, (128, 256)),
                   Inception_mod7(256*3, (144, 288)),
                   Inception_mod7(288*3, (160, 320)),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
    
    b5 = nn.Sequential(Inception_mod7(320*3, (160, 320)),
                   Inception_mod7(320*3, (192, 384)),
                   nn.AdaptiveAvgPool2d((1,1)),
                   nn.Flatten())

    net = nn.Sequential(b1, b2, b3, b4, b5, nn.Linear(384*3, num_labels))
    return net

googlenet3 = Googlenet_mod3(1, 10) 
LayerName = []
block_num = 0
incep_num = 0

for num, layer in googlenet3.named_children():  # 使用 named_children 来获取层名和层
    layername = layer.__class__.__name__
    # print(f"{num}: {layername}")  # 打印层名和层类名
    
    if layer.__class__.__name__ == 'Sequential':
        block_num += 1
        for sublayernum, sublayer in layer.named_children():  # 再次使用 named_children
            sublayername = sublayer.__class__.__name__
            if sublayername == 'Inception_mod5':
                incep_num += 1
            # print(f"  {sublayernum}: {sublayername}")
            layer_label = f'{layername[0]}{num}_{sublayername[0]}{sublayernum}'
            LayerName.append(layer_label)  # 收集子块的类型
                        
print('The layer name is:', LayerName)
print(f'The length of layer name is: {len(LayerName)}')
print('The number of blocks is:', block_num)
print('The number of inception blocks is:', incep_num)

The layer name is: ['S0_C0', 'S0_R1', 'S0_M2', 'S1_C0', 'S1_R1', 'S1_C2', 'S1_R3', 'S1_M4', 'S2_I0', 'S2_I1', 'S2_M2', 'S3_I0', 'S3_I1', 'S3_I2', 'S3_I3', 'S3_I4', 'S3_M5', 'S4_I0', 'S4_I1', 'S4_A2', 'S4_F3']
The length of layer name is: 21
The number of blocks is: 5
The number of inception blocks is: 0


In [72]:
googlenet_fmod3 = Googlenet_mod3(1, 10)
googlenet_cmod3 = Googlenet_mod3(3, 100)

googlenet_fmod3_modelinfo = []
googlenet_cmod3_modelinfo = []

macs_f, params_f = fashionmnist_info(googlenet_fmod3)   
print('*'*50)
macs_c, params_c = cifar100_info(googlenet_cmod3)

googlenet_fmod3_modelinfo = [macs_f, params_f]
googlenet_cmod3_modelinfo = [macs_c, params_c]

print('The model info of googlenet_fmod3 is:', googlenet_fmod3_modelinfo,'\n',
        'The model info of googlenet_cmod3 is:', googlenet_cmod3_modelinfo)

Sequential(
  11.71 M, 100.000% Params, 2.73 GMac, 99.771% MACs, 
  (0): Sequential(
    3.2 k, 0.027% Params, 41.75 MMac, 1.523% MACs, 
    (0): Conv2d(3.2 k, 0.027% Params, 40.14 MMac, 1.465% MACs, 1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
    (1): ReLU(0, 0.000% Params, 802.82 KMac, 0.029% MACs, )
    (2): MaxPool2d(0, 0.000% Params, 802.82 KMac, 0.029% MACs, kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  )
  (1): Sequential(
    114.94 k, 0.982% Params, 361.87 MMac, 13.206% MACs, 
    (0): Conv2d(4.16 k, 0.036% Params, 13.05 MMac, 0.476% MACs, 64, 64, kernel_size=(1, 1), stride=(1, 1))
    (1): ReLU(0, 0.000% Params, 200.7 KMac, 0.007% MACs, )
    (2): Conv2d(110.78 k, 0.946% Params, 347.42 MMac, 12.679% MACs, 64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(0, 0.000% Params, 602.11 KMac, 0.022% MACs, )
    (4): MaxPool2d(0, 0.000% Params, 602.11 KMac, 0.022% MACs, kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode

#### googlenet_mod4
all the inception block is the same as the route2, with only 4 line in the inception block

In [73]:
class Inception_mod8(nn.Module):
    # c1--c4是每条路径的输出通道数
    def __init__(self, in_channels, c2, **kwargs):
        super(Inception_mod8, self).__init__(**kwargs)
        # 线路1，单1x1卷积层
        self.p1_1 = nn.Conv2d(in_channels, c2[0], kernel_size=1)
        self.p1_2 = nn.Conv2d(c2[0], c2[1], kernel_size=3, padding=1)
        # 线路2，1x1卷积层后接3x3卷积层
        self.p2_1 = nn.Conv2d(in_channels, c2[0], kernel_size=1)
        self.p2_2 = nn.Conv2d(c2[0], c2[1], kernel_size=3, padding=1)
        # 线路3，1x1卷积层后接5x5卷积层
        self.p3_1 = nn.Conv2d(in_channels, c2[0], kernel_size=1)
        self.p3_2 = nn.Conv2d(c2[0], c2[1], kernel_size=3, padding=1)
        # 线路4，3x3最大汇聚层后接1x1卷积层
        self.p4_1 = nn.Conv2d(in_channels, c2[0], kernel_size=1)
        self.p4_2 = nn.Conv2d(c2[0], c2[1], kernel_size=3, padding=1)

    def forward(self, x):
        p1 = F.relu(self.p1_2(F.relu(self.p1_1(x))))
        p2 = F.relu(self.p2_2(F.relu(self.p2_1(x))))
        p3 = F.relu(self.p3_2(F.relu(self.p3_1(x))))
        p4 = F.relu(self.p4_2(F.relu(self.p4_1(x))))
        # 在通道维度上连结输出
        return torch.cat((p1, p2, p3, p4), dim=1)
    
def Googlenet_mod4(img_channel, num_labels):
    b1 = nn.Sequential(nn.Conv2d(img_channel, 64, kernel_size=7, stride=2, padding=3),
                   nn.ReLU(),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
    
    b2 = nn.Sequential(nn.Conv2d(64, 64, kernel_size=1),
                   nn.ReLU(),
                   nn.Conv2d(64, 192, kernel_size=3, padding=1),
                   nn.ReLU(),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
    
    b3 = nn.Sequential(Inception_mod8(192, (96, 128)),
                   Inception_mod8(128*4, (128, 192)),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
    
    b4 = nn.Sequential(Inception_mod8(192*4, (96, 208)),
                   Inception_mod8(208*4, (112, 224)),
                   Inception_mod8(224*4, (128, 256)),
                   Inception_mod8(256*4, (144, 288)),
                   Inception_mod8(288*4, (160, 320)),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
    
    b5 = nn.Sequential(Inception_mod8(320*4, (160, 320)),
                   Inception_mod8(320*4, (192, 384)),
                   nn.AdaptiveAvgPool2d((1,1)),
                   nn.Flatten())

    net = nn.Sequential(b1, b2, b3, b4, b5, nn.Linear(384*4, num_labels))
    return net

googlenet4 = Googlenet_mod4(1, 10) 
LayerName = []
block_num = 0
incep_num = 0

for num, layer in googlenet4.named_children():  # 使用 named_children 来获取层名和层
    layername = layer.__class__.__name__
    # print(f"{num}: {layername}")  # 打印层名和层类名
    
    if layer.__class__.__name__ == 'Sequential':
        block_num += 1
        for sublayernum, sublayer in layer.named_children():  # 再次使用 named_children
            sublayername = sublayer.__class__.__name__
            if sublayername == 'Inception_mod5':
                incep_num += 1
            # print(f"  {sublayernum}: {sublayername}")
            layer_label = f'{layername[0]}{num}_{sublayername[0]}{sublayernum}'
            LayerName.append(layer_label)  # 收集子块的类型
                        
print('The layer name is:', LayerName)
print(f'The length of layer name is: {len(LayerName)}')
print('The number of blocks is:', block_num)
print('The number of inception blocks is:', incep_num)

The layer name is: ['S0_C0', 'S0_R1', 'S0_M2', 'S1_C0', 'S1_R1', 'S1_C2', 'S1_R3', 'S1_M4', 'S2_I0', 'S2_I1', 'S2_M2', 'S3_I0', 'S3_I1', 'S3_I2', 'S3_I3', 'S3_I4', 'S3_M5', 'S4_I0', 'S4_I1', 'S4_A2', 'S4_F3']
The length of layer name is: 21
The number of blocks is: 5
The number of inception blocks is: 0


In [74]:
googlenet_fmod4 = Googlenet_mod4(1, 10)
googlenet_cmod4 = Googlenet_mod4(3, 100)

googlenet_fmod4_modelinfo = []
googlenet_cmod4_modelinfo = []

macs_f, params_f = fashionmnist_info(googlenet_fmod4)   
print('*'*50)
macs_c, params_c = cifar100_info(googlenet_cmod4)

googlenet_fmod4_modelinfo = [macs_f, params_f]
googlenet_cmod4_modelinfo = [macs_c, params_c]

print('The model info of googlenet_fmod4 is:', googlenet_fmod4_modelinfo,'\n',
        'The model info of googlenet_cmod4 is:', googlenet_cmod4_modelinfo)

Sequential(
  16.7 M, 100.000% Params, 3.7 GMac, 99.801% MACs, 
  (0): Sequential(
    3.2 k, 0.019% Params, 41.75 MMac, 1.125% MACs, 
    (0): Conv2d(3.2 k, 0.019% Params, 40.14 MMac, 1.081% MACs, 1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
    (1): ReLU(0, 0.000% Params, 802.82 KMac, 0.022% MACs, )
    (2): MaxPool2d(0, 0.000% Params, 802.82 KMac, 0.022% MACs, kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  )
  (1): Sequential(
    114.94 k, 0.688% Params, 361.87 MMac, 9.749% MACs, 
    (0): Conv2d(4.16 k, 0.025% Params, 13.05 MMac, 0.351% MACs, 64, 64, kernel_size=(1, 1), stride=(1, 1))
    (1): ReLU(0, 0.000% Params, 200.7 KMac, 0.005% MACs, )
    (2): Conv2d(110.78 k, 0.663% Params, 347.42 MMac, 9.360% MACs, 64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(0, 0.000% Params, 602.11 KMac, 0.016% MACs, )
    (4): MaxPool2d(0, 0.000% Params, 602.11 KMac, 0.016% MACs, kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=Fal

#### googlenet_mod5
all the inception block is the same as the route1, with only 1 line in the inception block

In [75]:
class Inception_mod5(nn.Module):
    # c1--c4是每条路径的输出通道数
    def __init__(self, in_channels, c1, **kwargs):
        super(Inception_mod5, self).__init__(**kwargs)
        # 线路1，单1x1卷积层
        self.p1_1 = nn.Conv2d(in_channels, c1, kernel_size=1)
        # 线路2，1x1卷积层后接3x3卷积层
        # self.p2_1 = nn.Conv2d(in_channels, c2[0], kernel_size=1)
        # self.p2_2 = nn.Conv2d(c2[0], c2[1], kernel_size=3, padding=1)
        # 线路3，1x1卷积层后接5x5卷积层
        # self.p3_1 = nn.Conv2d(in_channels, c3[0], kernel_size=1)
        # self.p3_2 = nn.Conv2d(c3[0], c3[1], kernel_size=5, padding=2)
        # 线路4，3x3最大汇聚层后接1x1卷积层
        # self.p4_1 = nn.MaxPool2d(kernel_size=3, stride=1, padding=1)
        # self.p4_2 = nn.Conv2d(in_channels, c4, kernel_size=1)

    def forward(self, x):
        p1 = F.relu(self.p1_1(x))
        # p2 = F.relu(self.p2_2(F.relu(self.p2_1(x))))
        # p3 = F.relu(self.p3_2(F.relu(self.p3_1(x))))
        # p4 = F.relu(self.p4_2(self.p4_1(x)))
        # 在通道维度上连结输出
        return torch.cat([p1], dim=1)
    
def Googlenet_mod5(img_channel, num_labels):
    b1 = nn.Sequential(nn.Conv2d(img_channel, 64, kernel_size=7, stride=2, padding=3),
                   nn.ReLU(),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
    
    b2 = nn.Sequential(nn.Conv2d(64, 64, kernel_size=1),
                   nn.ReLU(),
                   nn.Conv2d(64, 192, kernel_size=3, padding=1),
                   nn.ReLU(),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
    
    b3 = nn.Sequential(Inception_mod5(192, 64),
                   Inception_mod5(64, 128),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
    
    b4 = nn.Sequential(Inception_mod5(128, 192),
                   Inception_mod5(192, 160),
                   Inception_mod5(160, 128),
                   Inception_mod5(128, 112),
                   Inception_mod5(112, 256),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
    
    b5 = nn.Sequential(Inception_mod5(256, 256),
                   Inception_mod5(256, 384),
                   nn.AdaptiveAvgPool2d((1,1)),
                   nn.Flatten())

    net = nn.Sequential(b1, b2, b3, b4, b5, nn.Linear(384, num_labels))
    return net

googlenet5 = Googlenet_mod4(1, 10) 
LayerName = []
block_num = 0
incep_num = 0

for num, layer in googlenet5.named_children():  # 使用 named_children 来获取层名和层
    layername = layer.__class__.__name__
    # print(f"{num}: {layername}")  # 打印层名和层类名
    
    if layer.__class__.__name__ == 'Sequential':
        block_num += 1
        for sublayernum, sublayer in layer.named_children():  # 再次使用 named_children
            sublayername = sublayer.__class__.__name__
            if sublayername == 'Inception_mod5':
                incep_num += 1
            # print(f"  {sublayernum}: {sublayername}")
            layer_label = f'{layername[0]}{num}_{sublayername[0]}{sublayernum}'
            LayerName.append(layer_label)  # 收集子块的类型
                        
print('The layer name is:', LayerName)
print(f'The length of layer name is: {len(LayerName)}')
print('The number of blocks is:', block_num)
print('The number of inception blocks is:', incep_num)

The layer name is: ['S0_C0', 'S0_R1', 'S0_M2', 'S1_C0', 'S1_R1', 'S1_C2', 'S1_R3', 'S1_M4', 'S2_I0', 'S2_I1', 'S2_M2', 'S3_I0', 'S3_I1', 'S3_I2', 'S3_I3', 'S3_I4', 'S3_M5', 'S4_I0', 'S4_I1', 'S4_A2', 'S4_F3']
The length of layer name is: 21
The number of blocks is: 5
The number of inception blocks is: 0


In [76]:
googlenet_fmod5 = Googlenet_mod5(1, 10)
googlenet_cmod5 = Googlenet_mod5(3, 100)

googlenet_fmod5_modelinfo = []
googlenet_cmod5_modelinfo = []

macs_f, params_f = fashionmnist_info(googlenet_fmod5)   
print('*'*50)
macs_c, params_c = cifar100_info(googlenet_cmod5)

googlenet_fmod5_modelinfo = [macs_f, params_f]
googlenet_cmod5_modelinfo = [macs_c, params_c]

print('The model info of googlenet_fmod5 is:', googlenet_fmod5_modelinfo,'\n',
        'The model info of googlenet_cmod5 is:', googlenet_cmod5_modelinfo)

Sequential(
  426.78 k, 100.000% Params, 451.5 MMac, 99.225% MACs, 
  (0): Sequential(
    3.2 k, 0.750% Params, 41.75 MMac, 9.174% MACs, 
    (0): Conv2d(3.2 k, 0.750% Params, 40.14 MMac, 8.822% MACs, 1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
    (1): ReLU(0, 0.000% Params, 802.82 KMac, 0.176% MACs, )
    (2): MaxPool2d(0, 0.000% Params, 802.82 KMac, 0.176% MACs, kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  )
  (1): Sequential(
    114.94 k, 26.933% Params, 361.87 MMac, 79.526% MACs, 
    (0): Conv2d(4.16 k, 0.975% Params, 13.05 MMac, 2.867% MACs, 64, 64, kernel_size=(1, 1), stride=(1, 1))
    (1): ReLU(0, 0.000% Params, 200.7 KMac, 0.044% MACs, )
    (2): Conv2d(110.78 k, 25.958% Params, 347.42 MMac, 76.351% MACs, 64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(0, 0.000% Params, 602.11 KMac, 0.132% MACs, )
    (4): MaxPool2d(0, 0.000% Params, 602.11 KMac, 0.132% MACs, kernel_size=3, stride=2, padding=1, dilation=1, ceil_

#### googlenet_mod6
all the inception block is the same as the route1, with only 2 line in the inception block

In [77]:
class Inception_mod6(nn.Module):
    # c1--c4是每条路径的输出通道数
    def __init__(self, in_channels, c1, **kwargs):
        super(Inception_mod6, self).__init__(**kwargs)
        # 线路1，单1x1卷积层
        self.p1_1 = nn.Conv2d(in_channels, c1, kernel_size=1)
        # 线路2，1x1卷积层后接3x3卷积层
        self.p2_1 = nn.Conv2d(in_channels, c1, kernel_size=1)
        # self.p2_2 = nn.Conv2d(c2[0], c2[1], kernel_size=3, padding=1)
        # 线路3，1x1卷积层后接5x5卷积层
        # self.p3_1 = nn.Conv2d(in_channels, c2[0], kernel_size=1)
        # self.p3_2 = nn.Conv2d(c2[0], c2[1], kernel_size=3, padding=1)
        # 线路4，3x3最大汇聚层后接1x1卷积层
        # self.p4_1 = nn.MaxPool2d(kernel_size=3, stride=1, padding=1)
        # self.p4_2 = nn.Conv2d(in_channels, c4, kernel_size=1)

    def forward(self, x):
        p1 = F.relu(self.p1_1(x))
        p2 = F.relu(self.p2_1(x))
        # p2 = F.relu(self.p2_2(F.relu(self.p2_1(x))))
        # p3 = F.relu(self.p3_2(F.relu(self.p3_1(x))))
        # p4 = F.relu(self.p4_2(self.p4_1(x)))
        # 在通道维度上连结输出
        return torch.cat((p1, p2), dim=1)
    
def Googlenet_mod6(img_channel, num_labels):
    b1 = nn.Sequential(nn.Conv2d(img_channel, 64, kernel_size=7, stride=2, padding=3),
                   nn.ReLU(),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
    
    b2 = nn.Sequential(nn.Conv2d(64, 64, kernel_size=1),
                   nn.ReLU(),
                   nn.Conv2d(64, 192, kernel_size=3, padding=1),
                   nn.ReLU(),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
    
    b3 = nn.Sequential(Inception_mod6(192, 64),
                   Inception_mod6(128, 128),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
    
    b4 = nn.Sequential(Inception_mod6(256, 192),
                   Inception_mod6(384, 160),
                   Inception_mod6(320, 128),
                   Inception_mod6(256, 112),
                   Inception_mod6(224, 256),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
    
    b5 = nn.Sequential(Inception_mod6(512, 256),
                   Inception_mod6(512, 384),
                   nn.AdaptiveAvgPool2d((1,1)),
                   nn.Flatten())

    net = nn.Sequential(b1, b2, b3, b4, b5, nn.Linear(384*2, num_labels))
    return net

googlenet6 = Googlenet_mod6(1, 10) 
LayerName = []
block_num = 0
incep_num = 0

for num, layer in googlenet6.named_children():  # 使用 named_children 来获取层名和层
    layername = layer.__class__.__name__
    # print(f"{num}: {layername}")  # 打印层名和层类名
    
    if layer.__class__.__name__ == 'Sequential':
        block_num += 1
        for sublayernum, sublayer in layer.named_children():  # 再次使用 named_children
            sublayername = sublayer.__class__.__name__
            if sublayername == 'Inception_mod5':
                incep_num += 1
            # print(f"  {sublayernum}: {sublayername}")
            layer_label = f'{layername[0]}{num}_{sublayername[0]}{sublayernum}'
            LayerName.append(layer_label)  # 收集子块的类型
                        
print('The layer name is:', LayerName)
print(f'The length of layer name is: {len(LayerName)}')
print('The number of blocks is:', block_num)
print('The number of inception blocks is:', incep_num)

The layer name is: ['S0_C0', 'S0_R1', 'S0_M2', 'S1_C0', 'S1_R1', 'S1_C2', 'S1_R3', 'S1_M4', 'S2_I0', 'S2_I1', 'S2_M2', 'S3_I0', 'S3_I1', 'S3_I2', 'S3_I3', 'S3_I4', 'S3_M5', 'S4_I0', 'S4_I1', 'S4_A2', 'S4_F3']
The length of layer name is: 21
The number of blocks is: 5
The number of inception blocks is: 0


In [78]:
googlenet_fmod6 = Googlenet_mod6(1, 10)
googlenet_cmod6 = Googlenet_mod6(3, 100)

googlenet_fmod6_modelinfo = []
googlenet_cmod6_modelinfo = []

macs_f, params_f = fashionmnist_info(googlenet_fmod6)   
print('*'*50)
macs_c, params_c = cifar100_info(googlenet_cmod6)

googlenet_fmod6_modelinfo = [macs_f, params_f]
googlenet_cmod6_modelinfo = [macs_c, params_c]

print('The model info of googlenet_fmod6 is:', googlenet_fmod6_modelinfo,'\n',
        'The model info of googlenet_cmod6 is:', googlenet_cmod6_modelinfo)

Sequential(
  1.32 M, 100.000% Params, 574.86 MMac, 99.301% MACs, 
  (0): Sequential(
    3.2 k, 0.243% Params, 41.75 MMac, 7.211% MACs, 
    (0): Conv2d(3.2 k, 0.243% Params, 40.14 MMac, 6.934% MACs, 1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
    (1): ReLU(0, 0.000% Params, 802.82 KMac, 0.139% MACs, )
    (2): MaxPool2d(0, 0.000% Params, 802.82 KMac, 0.139% MACs, kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  )
  (1): Sequential(
    114.94 k, 8.727% Params, 361.87 MMac, 62.510% MACs, 
    (0): Conv2d(4.16 k, 0.316% Params, 13.05 MMac, 2.254% MACs, 64, 64, kernel_size=(1, 1), stride=(1, 1))
    (1): ReLU(0, 0.000% Params, 200.7 KMac, 0.035% MACs, )
    (2): Conv2d(110.78 k, 8.412% Params, 347.42 MMac, 60.014% MACs, 64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(0, 0.000% Params, 602.11 KMac, 0.104% MACs, )
    (4): MaxPool2d(0, 0.000% Params, 602.11 KMac, 0.104% MACs, kernel_size=3, stride=2, padding=1, dilation=1, ceil_mod

#### googlenet_mod7
all the inception block is the same as the route1, with only 3 line in the inception block

In [80]:
class Inception_mod7(nn.Module):
    # c1--c4是每条路径的输出通道数
    def __init__(self, in_channels, c1, **kwargs):
        super(Inception_mod7, self).__init__(**kwargs)
        # 线路1，单1x1卷积层
        self.p1_1 = nn.Conv2d(in_channels, c1, kernel_size=1)
        # 线路2，1x1卷积层后接3x3卷积层
        self.p2_1 = nn.Conv2d(in_channels, c1, kernel_size=1)
        # self.p2_2 = nn.Conv2d(c2[0], c2[1], kernel_size=3, padding=1)
        # 线路3，1x1卷积层后接5x5卷积层
        self.p3_1 = nn.Conv2d(in_channels, c1, kernel_size=1)
        # self.p3_2 = nn.Conv2d(c2[0], c2[1], kernel_size=3, padding=1)
        # 线路4，3x3最大汇聚层后接1x1卷积层
        # self.p4_1 = nn.Conv2d(in_channels, c2[0], kernel_size=1)
        # self.p4_2 = nn.Conv2d(c2[0], c2[1], kernel_size=3, padding=1)

    def forward(self, x):
        p1 = F.relu(self.p1_1(x))
        p2 = F.relu(self.p2_1(x))
        p3 = F.relu(self.p3_1(x))
        # p2 = F.relu(self.p2_2(F.relu(self.p2_1(x))))
        # p3 = F.relu(self.p3_2(F.relu(self.p3_1(x))))
        # p4 = F.relu(self.p4_2(F.relu(self.p4_1(x))))
        # 在通道维度上连结输出
        return torch.cat((p1,p2, p3), dim=1)
    
def Googlenet_mod7(img_channel, num_labels):
    b1 = nn.Sequential(nn.Conv2d(img_channel, 64, kernel_size=7, stride=2, padding=3),
                   nn.ReLU(),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
    
    b2 = nn.Sequential(nn.Conv2d(64, 64, kernel_size=1),
                   nn.ReLU(),
                   nn.Conv2d(64, 192, kernel_size=3, padding=1),
                   nn.ReLU(),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
    
    b3 = nn.Sequential(Inception_mod7(192, 64),
                   Inception_mod7(64*3, 128),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
    
    b4 = nn.Sequential(Inception_mod7(128*3, 192),
                   Inception_mod7(192*3, 160),
                   Inception_mod7(160*3, 128),
                   Inception_mod7(128*3, 112),
                   Inception_mod7(112*3, 256),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
    
    b5 = nn.Sequential(Inception_mod7(256*3, 256),
                   Inception_mod7(256*3, 384),
                   nn.AdaptiveAvgPool2d((1,1)),
                   nn.Flatten())

    net = nn.Sequential(b1, b2, b3, b4, b5, nn.Linear(384*3, num_labels))
    return net

googlenet7 = Googlenet_mod7(1, 10) 
LayerName = []
block_num = 0
incep_num = 0

for num, layer in googlenet7.named_children():  # 使用 named_children 来获取层名和层
    layername = layer.__class__.__name__
    # print(f"{num}: {layername}")  # 打印层名和层类名
    
    if layer.__class__.__name__ == 'Sequential':
        block_num += 1
        for sublayernum, sublayer in layer.named_children():  # 再次使用 named_children
            sublayername = sublayer.__class__.__name__
            if sublayername == 'Inception_mod5':
                incep_num += 1
            # print(f"  {sublayernum}: {sublayername}")
            layer_label = f'{layername[0]}{num}_{sublayername[0]}{sublayernum}'
            LayerName.append(layer_label)  # 收集子块的类型
                        
print('The layer name is:', LayerName)
print(f'The length of layer name is: {len(LayerName)}')
print('The number of blocks is:', block_num)
print('The number of inception blocks is:', incep_num)

The layer name is: ['S0_C0', 'S0_R1', 'S0_M2', 'S1_C0', 'S1_R1', 'S1_C2', 'S1_R3', 'S1_M4', 'S2_I0', 'S2_I1', 'S2_M2', 'S3_I0', 'S3_I1', 'S3_I2', 'S3_I3', 'S3_I4', 'S3_M5', 'S4_I0', 'S4_I1', 'S4_A2', 'S4_F3']
The length of layer name is: 21
The number of blocks is: 5
The number of inception blocks is: 0


In [81]:
googlenet_fmod7 = Googlenet_mod7(1, 10)
googlenet_cmod7 = Googlenet_mod7(3, 100)

googlenet_fmod7_modelinfo = []
googlenet_cmod7_modelinfo = []

macs_f, params_f = fashionmnist_info(googlenet_fmod7)   
print('*'*50)
macs_c, params_c = cifar100_info(googlenet_cmod7)

googlenet_fmod7_modelinfo = [macs_f, params_f]
googlenet_cmod7_modelinfo = [macs_c, params_c]

print('The model info of googlenet_fmod7 is:', googlenet_fmod7_modelinfo,'\n',
        'The model info of googlenet_cmod7 is:', googlenet_cmod7_modelinfo)

Sequential(
  2.79 M, 100.000% Params, 773.67 MMac, 99.414% MACs, 
  (0): Sequential(
    3.2 k, 0.115% Params, 41.75 MMac, 5.364% MACs, 
    (0): Conv2d(3.2 k, 0.115% Params, 40.14 MMac, 5.158% MACs, 1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
    (1): ReLU(0, 0.000% Params, 802.82 KMac, 0.103% MACs, )
    (2): MaxPool2d(0, 0.000% Params, 802.82 KMac, 0.103% MACs, kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  )
  (1): Sequential(
    114.94 k, 4.121% Params, 361.87 MMac, 46.499% MACs, 
    (0): Conv2d(4.16 k, 0.149% Params, 13.05 MMac, 1.676% MACs, 64, 64, kernel_size=(1, 1), stride=(1, 1))
    (1): ReLU(0, 0.000% Params, 200.7 KMac, 0.026% MACs, )
    (2): Conv2d(110.78 k, 3.972% Params, 347.42 MMac, 44.642% MACs, 64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(0, 0.000% Params, 602.11 KMac, 0.077% MACs, )
    (4): MaxPool2d(0, 0.000% Params, 602.11 KMac, 0.077% MACs, kernel_size=3, stride=2, padding=1, dilation=1, ceil_mod

#### googlenet_mod8
all the inception block is the same as the route1, with only 4 line in the inception block

In [82]:
class Inception_mod8(nn.Module):
    # c1--c4是每条路径的输出通道数
    def __init__(self, in_channels, c1, **kwargs):
        super(Inception_mod8, self).__init__(**kwargs)
        # 线路1，单1x1卷积层
        self.p1_1 = nn.Conv2d(in_channels, c1, kernel_size=1)
        # 线路2，1x1卷积层后接3x3卷积层
        self.p2_1 = nn.Conv2d(in_channels, c1, kernel_size=1)
        # self.p2_2 = nn.Conv2d(c2[0], c2[1], kernel_size=3, padding=1)
        # 线路3，1x1卷积层后接5x5卷积层
        self.p3_1 = nn.Conv2d(in_channels, c1, kernel_size=1)
        # self.p3_2 = nn.Conv2d(c2[0], c2[1], kernel_size=3, padding=1)
        # 线路4，3x3最大汇聚层后接1x1卷积层
        self.p4_1 = nn.Conv2d(in_channels, c1, kernel_size=1)
        # self.p4_2 = nn.Conv2d(c2[0], c2[1], kernel_size=3, padding=1)

    def forward(self, x):
        p1 = F.relu(self.p1_1(x))
        p2 = F.relu(self.p2_1(x))
        p3 = F.relu(self.p3_1(x))
        p4 = F.relu(self.p4_1(x))
        # 在通道维度上连结输出
        return torch.cat((p1, p2, p3, p4), dim=1)
    
def Googlenet_mod8(img_channel, num_labels):
    b1 = nn.Sequential(nn.Conv2d(img_channel, 64, kernel_size=7, stride=2, padding=3),
                   nn.ReLU(),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
    
    b2 = nn.Sequential(nn.Conv2d(64, 64, kernel_size=1),
                   nn.ReLU(),
                   nn.Conv2d(64, 192, kernel_size=3, padding=1),
                   nn.ReLU(),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
    
    b3 = nn.Sequential(Inception_mod8(192, 64),
                   Inception_mod8(64*4, 128),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
    
    b4 = nn.Sequential(Inception_mod8(128*4, 192),
                   Inception_mod8(192*4, 160),
                   Inception_mod8(160*4, 128),
                   Inception_mod8(128*4, 112),
                   Inception_mod8(112*4, 256),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
    
    b5 = nn.Sequential(Inception_mod8(256*4, 256),
                   Inception_mod8(256*4, 384),
                   nn.AdaptiveAvgPool2d((1,1)),
                   nn.Flatten())

    net = nn.Sequential(b1, b2, b3, b4, b5, nn.Linear(384*4, num_labels))
    return net

googlenet8 = Googlenet_mod8(1, 10) 
LayerName = []
block_num = 0
incep_num = 0

for num, layer in googlenet8.named_children():  # 使用 named_children 来获取层名和层
    layername = layer.__class__.__name__
    # print(f"{num}: {layername}")  # 打印层名和层类名
    
    if layer.__class__.__name__ == 'Sequential':
        block_num += 1
        for sublayernum, sublayer in layer.named_children():  # 再次使用 named_children
            sublayername = sublayer.__class__.__name__
            if sublayername == 'Inception_mod5':
                incep_num += 1
            # print(f"  {sublayernum}: {sublayername}")
            layer_label = f'{layername[0]}{num}_{sublayername[0]}{sublayernum}'
            LayerName.append(layer_label)  # 收集子块的类型
                        
print('The layer name is:', LayerName)
print(f'The length of layer name is: {len(LayerName)}')
print('The number of blocks is:', block_num)
print('The number of inception blocks is:', incep_num)

The layer name is: ['S0_C0', 'S0_R1', 'S0_M2', 'S1_C0', 'S1_R1', 'S1_C2', 'S1_R3', 'S1_M4', 'S2_I0', 'S2_I1', 'S2_M2', 'S3_I0', 'S3_I1', 'S3_I2', 'S3_I3', 'S3_I4', 'S3_M5', 'S4_I0', 'S4_I1', 'S4_A2', 'S4_F3']
The length of layer name is: 21
The number of blocks is: 5
The number of inception blocks is: 0


In [83]:
googlenet_fmod8 = Googlenet_mod8(1, 10)
googlenet_cmod8 = Googlenet_mod8(3, 100)

googlenet_fmod8_modelinfo = []
googlenet_cmod8_modelinfo = []

macs_f, params_f = fashionmnist_info(googlenet_fmod8)   
print('*'*50)
macs_c, params_c = cifar100_info(googlenet_cmod8)

googlenet_fmod8_modelinfo = [macs_f, params_f]
googlenet_cmod8_modelinfo = [macs_c, params_c]

print('The model info of googlenet_fmod8 is:', googlenet_fmod8_modelinfo,'\n',
        'The model info of googlenet_cmod8 is:', googlenet_cmod8_modelinfo)

Sequential(
  4.84 M, 100.000% Params, 1.05 GMac, 99.518% MACs, 
  (0): Sequential(
    3.2 k, 0.066% Params, 41.75 MMac, 3.964% MACs, 
    (0): Conv2d(3.2 k, 0.066% Params, 40.14 MMac, 3.812% MACs, 1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
    (1): ReLU(0, 0.000% Params, 802.82 KMac, 0.076% MACs, )
    (2): MaxPool2d(0, 0.000% Params, 802.82 KMac, 0.076% MACs, kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  )
  (1): Sequential(
    114.94 k, 2.374% Params, 361.87 MMac, 34.364% MACs, 
    (0): Conv2d(4.16 k, 0.086% Params, 13.05 MMac, 1.239% MACs, 64, 64, kernel_size=(1, 1), stride=(1, 1))
    (1): ReLU(0, 0.000% Params, 200.7 KMac, 0.019% MACs, )
    (2): Conv2d(110.78 k, 2.288% Params, 347.42 MMac, 32.992% MACs, 64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(0, 0.000% Params, 602.11 KMac, 0.057% MACs, )
    (4): MaxPool2d(0, 0.000% Params, 602.11 KMac, 0.057% MACs, kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=

#### googlenet_mod9
all the inception block is the same as the route3, with only 1 line in the inception block

In [84]:
class Inception_mod5(nn.Module):
    # c1--c4是每条路径的输出通道数
    def __init__(self, in_channels, c3, **kwargs):
        super(Inception_mod5, self).__init__(**kwargs)
        # 线路1，单1x1卷积层
        # self.p1_1 = nn.Conv2d(in_channels, c1, kernel_size=1)
        # 线路2，1x1卷积层后接3x3卷积层
        # self.p2_1 = nn.Conv2d(in_channels, c2[0], kernel_size=1)
        # self.p2_2 = nn.Conv2d(c2[0], c2[1], kernel_size=3, padding=1)
        # 线路3，1x1卷积层后接5x5卷积层
        self.p3_1 = nn.Conv2d(in_channels, c3[0], kernel_size=1)
        self.p3_2 = nn.Conv2d(c3[0], c3[1], kernel_size=5, padding=2)
        # 线路4，3x3最大汇聚层后接1x1卷积层
        # self.p4_1 = nn.MaxPool2d(kernel_size=3, stride=1, padding=1)
        # self.p4_2 = nn.Conv2d(in_channels, c4, kernel_size=1)

    def forward(self, x):
        # p1 = F.relu(self.p1_1(x))
        # p2 = F.relu(self.p2_2(F.relu(self.p2_1(x))))
        p3 = F.relu(self.p3_2(F.relu(self.p3_1(x))))
        # p4 = F.relu(self.p4_2(self.p4_1(x)))
        # 在通道维度上连结输出
        return torch.cat([p3], dim=1)
    
def Googlenet_mod9(img_channel, num_labels):
    b1 = nn.Sequential(nn.Conv2d(img_channel, 64, kernel_size=7, stride=2, padding=3),
                   nn.ReLU(),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
    
    b2 = nn.Sequential(nn.Conv2d(64, 64, kernel_size=1),
                   nn.ReLU(),
                   nn.Conv2d(64, 192, kernel_size=3, padding=1),
                   nn.ReLU(),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
    
    b3 = nn.Sequential(Inception_mod5(192, (16, 32)),
                   Inception_mod5(32, (32, 96)),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
    
    b4 = nn.Sequential(Inception_mod5(96, (16, 48)),
                   Inception_mod5(48,(24, 64)),
                   Inception_mod5(64, (24, 64)),
                   Inception_mod5(64, (32, 64)),
                   Inception_mod5(64, (32, 128)),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
    
    b5 = nn.Sequential(Inception_mod5(128, (32, 128)),
                   Inception_mod5(128, (48, 128)),
                   nn.AdaptiveAvgPool2d((1,1)),
                   nn.Flatten())

    net = nn.Sequential(b1, b2, b3, b4, b5, nn.Linear(128, num_labels))
    return net

googlenet9 = Googlenet_mod9(1, 10) 
LayerName = []
block_num = 0
incep_num = 0

for num, layer in googlenet9.named_children():  # 使用 named_children 来获取层名和层
    layername = layer.__class__.__name__
    # print(f"{num}: {layername}")  # 打印层名和层类名
    
    if layer.__class__.__name__ == 'Sequential':
        block_num += 1
        for sublayernum, sublayer in layer.named_children():  # 再次使用 named_children
            sublayername = sublayer.__class__.__name__
            if sublayername == 'Inception_mod5':
                incep_num += 1
            # print(f"  {sublayernum}: {sublayername}")
            layer_label = f'{layername[0]}{num}_{sublayername[0]}{sublayernum}'
            LayerName.append(layer_label)  # 收集子块的类型
                        
print('The layer name is:', LayerName)
print(f'The length of layer name is: {len(LayerName)}')
print('The number of blocks is:', block_num)
print('The number of inception blocks is:', incep_num)

The layer name is: ['S0_C0', 'S0_R1', 'S0_M2', 'S1_C0', 'S1_R1', 'S1_C2', 'S1_R3', 'S1_M4', 'S2_I0', 'S2_I1', 'S2_M2', 'S3_I0', 'S3_I1', 'S3_I2', 'S3_I3', 'S3_I4', 'S3_M5', 'S4_I0', 'S4_I1', 'S4_A2', 'S4_F3']
The length of layer name is: 21
The number of blocks is: 5
The number of inception blocks is: 9


In [85]:
googlenet_fmod9 = Googlenet_mod9(1, 10)
googlenet_cmod9 = Googlenet_mod9(3, 100)

googlenet_fmod9_modelinfo = []
googlenet_cmod9_modelinfo = []

macs_f, params_f = fashionmnist_info(googlenet_fmod9)   
print('*'*50)
macs_c, params_c = cifar100_info(googlenet_cmod9)

googlenet_fmod9_modelinfo = [macs_f, params_f]
googlenet_cmod9_modelinfo = [macs_c, params_c]

print('The model info of googlenet_fmod9 is:', googlenet_fmod9_modelinfo,'\n',
        'The model info of googlenet_cmod9 is:', googlenet_cmod9_modelinfo)

Sequential(
  738.3 k, 100.000% Params, 541.03 MMac, 99.381% MACs, 
  (0): Sequential(
    3.2 k, 0.433% Params, 41.75 MMac, 7.668% MACs, 
    (0): Conv2d(3.2 k, 0.433% Params, 40.14 MMac, 7.373% MACs, 1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
    (1): ReLU(0, 0.000% Params, 802.82 KMac, 0.147% MACs, )
    (2): MaxPool2d(0, 0.000% Params, 802.82 KMac, 0.147% MACs, kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  )
  (1): Sequential(
    114.94 k, 15.569% Params, 361.87 MMac, 66.471% MACs, 
    (0): Conv2d(4.16 k, 0.563% Params, 13.05 MMac, 2.396% MACs, 64, 64, kernel_size=(1, 1), stride=(1, 1))
    (1): ReLU(0, 0.000% Params, 200.7 KMac, 0.037% MACs, )
    (2): Conv2d(110.78 k, 15.005% Params, 347.42 MMac, 63.817% MACs, 64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(0, 0.000% Params, 602.11 KMac, 0.111% MACs, )
    (4): MaxPool2d(0, 0.000% Params, 602.11 KMac, 0.111% MACs, kernel_size=3, stride=2, padding=1, dilation=1, ceil_

#### googlenet_mod10
all the inception block is the same as the route3, with only 2 line in the inception block

In [86]:
class Inception_mod6(nn.Module):
    # c1--c4是每条路径的输出通道数
    def __init__(self, in_channels, c3, **kwargs):
        super(Inception_mod6, self).__init__(**kwargs)
        # 线路1，单1x1卷积层
        # self.p1_1 = nn.Conv2d(in_channels, c1, kernel_size=1)
        # 线路2，1x1卷积层后接3x3卷积层
        self.p2_1 = nn.Conv2d(in_channels, c3[0], kernel_size=1)
        self.p2_2 = nn.Conv2d(c3[0], c3[1], kernel_size=5, padding=2)
        # 线路3，1x1卷积层后接5x5卷积层
        self.p3_1 = nn.Conv2d(in_channels, c3[0], kernel_size=1)
        self.p3_2 = nn.Conv2d(c3[0], c3[1], kernel_size=5, padding=2)
        # 线路4，3x3最大汇聚层后接1x1卷积层
        # self.p4_1 = nn.MaxPool2d(kernel_size=3, stride=1, padding=1)
        # self.p4_2 = nn.Conv2d(in_channels, c4, kernel_size=1)

    def forward(self, x):
        # p1 = F.relu(self.p1_1(x))
        # p2 = F.relu(self.p2_1(x))
        p2 = F.relu(self.p2_2(F.relu(self.p2_1(x))))
        p3 = F.relu(self.p3_2(F.relu(self.p3_1(x))))
        # p4 = F.relu(self.p4_2(self.p4_1(x)))
        # 在通道维度上连结输出
        return torch.cat((p2, p3), dim=1)
    
def Googlenet_mod10(img_channel, num_labels):
    b1 = nn.Sequential(nn.Conv2d(img_channel, 64, kernel_size=7, stride=2, padding=3),
                   nn.ReLU(),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
    
    b2 = nn.Sequential(nn.Conv2d(64, 64, kernel_size=1),
                   nn.ReLU(),
                   nn.Conv2d(64, 192, kernel_size=3, padding=1),
                   nn.ReLU(),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
    
    b3 = nn.Sequential(Inception_mod6(192, (16, 32)),
                   Inception_mod6(64, (32, 96)),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
    
    b4 = nn.Sequential(Inception_mod6(96*2, (16, 48)),
                   Inception_mod6(48*2, (24, 64)),
                   Inception_mod6(64*2, (24, 64)),
                   Inception_mod6(64*2, (32, 64)),
                   Inception_mod6(64*2, (32, 128)),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
    
    b5 = nn.Sequential(Inception_mod6(128*2, (32, 128)),
                   Inception_mod6(128*2, (48, 128)),
                   nn.AdaptiveAvgPool2d((1,1)),
                   nn.Flatten())

    net = nn.Sequential(b1, b2, b3, b4, b5, nn.Linear(128*2, num_labels))
    return net

googlenet10 = Googlenet_mod10(1, 10) 
LayerName = []
block_num = 0
incep_num = 0

for num, layer in googlenet10.named_children():  # 使用 named_children 来获取层名和层
    layername = layer.__class__.__name__
    # print(f"{num}: {layername}")  # 打印层名和层类名
    
    if layer.__class__.__name__ == 'Sequential':
        block_num += 1
        for sublayernum, sublayer in layer.named_children():  # 再次使用 named_children
            sublayername = sublayer.__class__.__name__
            if sublayername == 'Inception_mod5':
                incep_num += 1
            # print(f"  {sublayernum}: {sublayername}")
            layer_label = f'{layername[0]}{num}_{sublayername[0]}{sublayernum}'
            LayerName.append(layer_label)  # 收集子块的类型
                        
print('The layer name is:', LayerName)
print(f'The length of layer name is: {len(LayerName)}')
print('The number of blocks is:', block_num)
print('The number of inception blocks is:', incep_num)

The layer name is: ['S0_C0', 'S0_R1', 'S0_M2', 'S1_C0', 'S1_R1', 'S1_C2', 'S1_R3', 'S1_M4', 'S2_I0', 'S2_I1', 'S2_M2', 'S3_I0', 'S3_I1', 'S3_I2', 'S3_I3', 'S3_I4', 'S3_M5', 'S4_I0', 'S4_I1', 'S4_A2', 'S4_F3']
The length of layer name is: 21
The number of blocks is: 5
The number of inception blocks is: 0


In [87]:
googlenet_fmod10 = Googlenet_mod10(1, 10)
googlenet_cmod10 = Googlenet_mod10(3, 100)

googlenet_fmod10_modelinfo = []
googlenet_cmod10_modelinfo = []

macs_f, params_f = fashionmnist_info(googlenet_fmod10)   
print('*'*50)
macs_c, params_c = cifar100_info(googlenet_cmod10)

googlenet_fmod10_modelinfo = [macs_f, params_f]
googlenet_cmod10_modelinfo = [macs_c, params_c]

print('The model info of googlenet_fmod10 is:', googlenet_fmod10_modelinfo,'\n',
        'The model info of googlenet_cmod10 is:', googlenet_cmod10_modelinfo)

Sequential(
  1.4 M, 100.000% Params, 684.32 MMac, 99.458% MACs, 
  (0): Sequential(
    3.2 k, 0.229% Params, 41.75 MMac, 6.067% MACs, 
    (0): Conv2d(3.2 k, 0.229% Params, 40.14 MMac, 5.834% MACs, 1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
    (1): ReLU(0, 0.000% Params, 802.82 KMac, 0.117% MACs, )
    (2): MaxPool2d(0, 0.000% Params, 802.82 KMac, 0.117% MACs, kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  )
  (1): Sequential(
    114.94 k, 8.224% Params, 361.87 MMac, 52.594% MACs, 
    (0): Conv2d(4.16 k, 0.298% Params, 13.05 MMac, 1.896% MACs, 64, 64, kernel_size=(1, 1), stride=(1, 1))
    (1): ReLU(0, 0.000% Params, 200.7 KMac, 0.029% MACs, )
    (2): Conv2d(110.78 k, 7.927% Params, 347.42 MMac, 50.494% MACs, 64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(0, 0.000% Params, 602.11 KMac, 0.088% MACs, )
    (4): MaxPool2d(0, 0.000% Params, 602.11 KMac, 0.088% MACs, kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode

#### googlenet_mod11
all the inception block is the same as the route3, with only 3 line in the inception block

In [88]:
class Inception_mod7(nn.Module):
    # c1--c4是每条路径的输出通道数
    def __init__(self, in_channels, c3, **kwargs):
        super(Inception_mod7, self).__init__(**kwargs)
        # 线路1，单1x1卷积层
        self.p1_1 = nn.Conv2d(in_channels, c3[0], kernel_size=1)
        self.p1_2 = nn.Conv2d(c3[0], c3[1], kernel_size=5, padding=2)
        # 线路2，1x1卷积层后接3x3卷积层
        self.p2_1 = nn.Conv2d(in_channels, c3[0], kernel_size=1)
        self.p2_2 = nn.Conv2d(c3[0], c3[1], kernel_size=5, padding=2)
        # 线路3，1x1卷积层后接5x5卷积层
        self.p3_1 = nn.Conv2d(in_channels, c3[0], kernel_size=1)
        self.p3_2 = nn.Conv2d(c3[0], c3[1], kernel_size=5, padding=2)
        # 线路4，3x3最大汇聚层后接1x1卷积层
        # self.p4_1 = nn.Conv2d(in_channels, c2[0], kernel_size=1)
        # self.p4_2 = nn.Conv2d(c2[0], c2[1], kernel_size=3, padding=1)

    def forward(self, x):
        # p1 = F.relu(self.p1_1(x))
        # p2 = F.relu(self.p2_1(x))
        # p3 = F.relu(self.p3_1(x))
        p1 = F.relu(self.p1_2(F.relu(self.p1_1(x))))
        p2 = F.relu(self.p2_2(F.relu(self.p2_1(x))))
        p3 = F.relu(self.p3_2(F.relu(self.p3_1(x))))
        # p4 = F.relu(self.p4_2(F.relu(self.p4_1(x))))
        # 在通道维度上连结输出
        return torch.cat((p1,p2, p3), dim=1)
    
def Googlenet_mod11(img_channel, num_labels):
    b1 = nn.Sequential(nn.Conv2d(img_channel, 64, kernel_size=7, stride=2, padding=3),
                   nn.ReLU(),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
    
    b2 = nn.Sequential(nn.Conv2d(64, 64, kernel_size=1),
                   nn.ReLU(),
                   nn.Conv2d(64, 192, kernel_size=3, padding=1),
                   nn.ReLU(),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
    
    b3 = nn.Sequential(Inception_mod7(192, (16, 32)),
                   Inception_mod7(32*3, (32, 96)),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
    
    b4 = nn.Sequential(Inception_mod7(96*3, (16, 48)),
                   Inception_mod7(48*3, (24, 64)),
                   Inception_mod7(64*3, (24, 64)),
                   Inception_mod7(64*3, (32, 64)),
                   Inception_mod7(64*3, (32, 128)),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
    
    b5 = nn.Sequential(Inception_mod7(128*3, (32, 128)),
                   Inception_mod7(128*3, (48, 128)),
                   nn.AdaptiveAvgPool2d((1,1)),
                   nn.Flatten())

    net = nn.Sequential(b1, b2, b3, b4, b5, nn.Linear(128*3, num_labels))
    return net

googlenet11 = Googlenet_mod11(1, 10) 
LayerName = []
block_num = 0
incep_num = 0

for num, layer in googlenet11.named_children():  # 使用 named_children 来获取层名和层
    layername = layer.__class__.__name__
    # print(f"{num}: {layername}")  # 打印层名和层类名
    
    if layer.__class__.__name__ == 'Sequential':
        block_num += 1
        for sublayernum, sublayer in layer.named_children():  # 再次使用 named_children
            sublayername = sublayer.__class__.__name__
            if sublayername == 'Inception_mod5':
                incep_num += 1
            # print(f"  {sublayernum}: {sublayername}")
            layer_label = f'{layername[0]}{num}_{sublayername[0]}{sublayernum}'
            LayerName.append(layer_label)  # 收集子块的类型
                        
print('The layer name is:', LayerName)
print(f'The length of layer name is: {len(LayerName)}')
print('The number of blocks is:', block_num)
print('The number of inception blocks is:', incep_num)

The layer name is: ['S0_C0', 'S0_R1', 'S0_M2', 'S1_C0', 'S1_R1', 'S1_C2', 'S1_R3', 'S1_M4', 'S2_I0', 'S2_I1', 'S2_M2', 'S3_I0', 'S3_I1', 'S3_I2', 'S3_I3', 'S3_I4', 'S3_M5', 'S4_I0', 'S4_I1', 'S4_A2', 'S4_F3']
The length of layer name is: 21
The number of blocks is: 5
The number of inception blocks is: 0


In [89]:
googlenet_fmod11 = Googlenet_mod11(1, 10)
googlenet_cmod11 = Googlenet_mod11(3, 100)

googlenet_fmod11_modelinfo = []
googlenet_cmod11_modelinfo = []

macs_f, params_f = fashionmnist_info(googlenet_fmod11)   
print('*'*50)
macs_c, params_c = cifar100_info(googlenet_cmod11)

googlenet_fmod11_modelinfo = [macs_f, params_f]
googlenet_cmod11_modelinfo = [macs_c, params_c]

print('The model info of googlenet_fmod11 is:', googlenet_fmod11_modelinfo,'\n',
        'The model info of googlenet_cmod11 is:', googlenet_cmod11_modelinfo)

Sequential(
  2.1 M, 100.000% Params, 833.47 MMac, 99.512% MACs, 
  (0): Sequential(
    3.2 k, 0.153% Params, 41.75 MMac, 4.984% MACs, 
    (0): Conv2d(3.2 k, 0.153% Params, 40.14 MMac, 4.793% MACs, 1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
    (1): ReLU(0, 0.000% Params, 802.82 KMac, 0.096% MACs, )
    (2): MaxPool2d(0, 0.000% Params, 802.82 KMac, 0.096% MACs, kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  )
  (1): Sequential(
    114.94 k, 5.484% Params, 361.87 MMac, 43.205% MACs, 
    (0): Conv2d(4.16 k, 0.198% Params, 13.05 MMac, 1.558% MACs, 64, 64, kernel_size=(1, 1), stride=(1, 1))
    (1): ReLU(0, 0.000% Params, 200.7 KMac, 0.024% MACs, )
    (2): Conv2d(110.78 k, 5.285% Params, 347.42 MMac, 41.480% MACs, 64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(0, 0.000% Params, 602.11 KMac, 0.072% MACs, )
    (4): MaxPool2d(0, 0.000% Params, 602.11 KMac, 0.072% MACs, kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode

#### googlenet_mod12
all the inception block is the same as the route3, with only 4 line in the inception block

In [94]:
class Inception_mod8(nn.Module):
    # c1--c4是每条路径的输出通道数
    def __init__(self, in_channels, c3, **kwargs):
        super(Inception_mod8, self).__init__(**kwargs)
        # 线路3，1x1卷积层后接5x5卷积层
        self.p1_1 = nn.Conv2d(in_channels, c3[0], kernel_size=1)
        self.p1_2 = nn.Conv2d(c3[0], c3[1], kernel_size=5, padding=2)
        # 线路3，1x1卷积层后接5x5卷积层
        self.p2_1 = nn.Conv2d(in_channels, c3[0], kernel_size=1)
        self.p2_2 = nn.Conv2d(c3[0], c3[1], kernel_size=5, padding=2)
        # 线路3，1x1卷积层后接5x5卷积层
        self.p3_1 = nn.Conv2d(in_channels, c3[0], kernel_size=1)
        self.p3_2 = nn.Conv2d(c3[0], c3[1], kernel_size=5, padding=2)
        # 线路3，1x1卷积层后接5x5卷积层
        self.p4_1 = nn.Conv2d(in_channels, c3[0], kernel_size=1)
        self.p4_2 = nn.Conv2d(c3[0], c3[1], kernel_size=5, padding=2)

    def forward(self, x):
        p1 = F.relu(self.p1_2(F.relu(self.p1_1(x))))
        p2 = F.relu(self.p2_2(F.relu(self.p2_1(x))))
        p3 = F.relu(self.p3_2(F.relu(self.p3_1(x))))
        p4 = F.relu(self.p4_2(F.relu(self.p4_1(x))))
        # 在通道维度上连结输出
        return torch.cat((p1, p2, p3, p4), dim=1)
    
def Googlenet_mod12(img_channel, num_labels):
    b1 = nn.Sequential(nn.Conv2d(img_channel, 64, kernel_size=7, stride=2, padding=3),
                   nn.ReLU(),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
    
    b2 = nn.Sequential(nn.Conv2d(64, 64, kernel_size=1),
                   nn.ReLU(),
                   nn.Conv2d(64, 192, kernel_size=3, padding=1),
                   nn.ReLU(),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
    
    b3 = nn.Sequential(Inception_mod8(192, (16, 32)),
                   Inception_mod8(32*4, (32, 96)),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
    
    b4 = nn.Sequential(Inception_mod8(96*4, (16, 48)),
                   Inception_mod8(48*4, (24, 64)),
                   Inception_mod8(64*4, (24, 64)),
                   Inception_mod8(64*4, (32, 64)),
                   Inception_mod8(64*4, (32, 128)),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
    
    b5 = nn.Sequential(Inception_mod8(128*4, (32, 128)),
                   Inception_mod8(128*4, (48, 128)),
                   nn.AdaptiveAvgPool2d((1,1)),
                   nn.Flatten())

    net = nn.Sequential(b1, b2, b3, b4, b5, nn.Linear(128*4, num_labels))
    return net

googlenet12 = Googlenet_mod12(1, 10) 
LayerName = []
block_num = 0
incep_num = 0

for num, layer in googlenet12.named_children():  # 使用 named_children 来获取层名和层
    layername = layer.__class__.__name__
    # print(f"{num}: {layername}")  # 打印层名和层类名
    
    if layer.__class__.__name__ == 'Sequential':
        block_num += 1
        for sublayernum, sublayer in layer.named_children():  # 再次使用 named_children
            sublayername = sublayer.__class__.__name__
            if sublayername == 'Inception_mod5':
                incep_num += 1
            # print(f"  {sublayernum}: {sublayername}")
            layer_label = f'{layername[0]}{num}_{sublayername[0]}{sublayernum}'
            LayerName.append(layer_label)  # 收集子块的类型
                        
print('The layer name is:', LayerName)
print(f'The length of layer name is: {len(LayerName)}')
print('The number of blocks is:', block_num)
print('The number of inception blocks is:', incep_num)

The layer name is: ['S0_C0', 'S0_R1', 'S0_M2', 'S1_C0', 'S1_R1', 'S1_C2', 'S1_R3', 'S1_M4', 'S2_I0', 'S2_I1', 'S2_M2', 'S3_I0', 'S3_I1', 'S3_I2', 'S3_I3', 'S3_I4', 'S3_M5', 'S4_I0', 'S4_I1', 'S4_A2', 'S4_F3']
The length of layer name is: 21
The number of blocks is: 5
The number of inception blocks is: 0


In [95]:
googlenet_fmod12 = Googlenet_mod12(1, 10)
googlenet_cmod12 = Googlenet_mod12(3, 100)

googlenet_fmod12_modelinfo = []
googlenet_cmod12_modelinfo = []

macs_f, params_f = fashionmnist_info(googlenet_fmod12)   
print('*'*50)
macs_c, params_c = cifar100_info(googlenet_cmod12)

googlenet_fmod12_modelinfo = [macs_f, params_f]
googlenet_cmod12_modelinfo = [macs_c, params_c]

print('The model info of googlenet_fmod12 is:', googlenet_fmod12_modelinfo,'\n',
        'The model info of googlenet_cmod12 is:', googlenet_cmod12_modelinfo)

Sequential(
  2.83 M, 100.000% Params, 988.5 MMac, 99.552% MACs, 
  (0): Sequential(
    3.2 k, 0.113% Params, 41.75 MMac, 4.204% MACs, 
    (0): Conv2d(3.2 k, 0.113% Params, 40.14 MMac, 4.043% MACs, 1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
    (1): ReLU(0, 0.000% Params, 802.82 KMac, 0.081% MACs, )
    (2): MaxPool2d(0, 0.000% Params, 802.82 KMac, 0.081% MACs, kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  )
  (1): Sequential(
    114.94 k, 4.056% Params, 361.87 MMac, 36.444% MACs, 
    (0): Conv2d(4.16 k, 0.147% Params, 13.05 MMac, 1.314% MACs, 64, 64, kernel_size=(1, 1), stride=(1, 1))
    (1): ReLU(0, 0.000% Params, 200.7 KMac, 0.020% MACs, )
    (2): Conv2d(110.78 k, 3.909% Params, 347.42 MMac, 34.989% MACs, 64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(0, 0.000% Params, 602.11 KMac, 0.061% MACs, )
    (4): MaxPool2d(0, 0.000% Params, 602.11 KMac, 0.061% MACs, kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode